In [ ]:
pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.


In [ ]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.0 MB/s eta 0:00:00


In [ ]:

import os
import sys
import evaluate
import argparse
import random
import yaml
from tqdm import tqdm
import warnings
import json
from sklearn.model_selection import train_test_split
import gc
import re
import traceback

# CRITICAL: Set environment variables BEFORE importing PyTorch
os.environ["FORCE_TORCH_SDPA_KERNEL"] = "0"
os.environ["PYTORCH_DISABLE_SDPA"] = "1"
os.environ["PYTORCH_NO_CUDA_MEMORY_EFFICIENCY_WARNING"] = "1"

# Data Handling
import numpy as np
import pandas as pd
import h5py

# Deep Learning - PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Transformers Library
from transformers import (
    AutoTokenizer,
    MBartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    set_seed as transformers_set_seed,
    GenerationConfig
)

# Evaluation Metric
import sacrebleu

# Google Drive
try:
    from google.colab import drive
except ImportError:
    print("Not running in Google Colab environment.")

# For custom collator
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Union
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.utils import PaddingStrategy

# === Configuration & Argument Parsing ===
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# === Global Variables ===
global_tokenizer = None  # Will be set after tokenizer initialization

In [ ]:



def parse_args():
    parser = argparse.ArgumentParser(description="Train mBART model for Saudi Sign Language Translation")

    # Update paths for Colab environment
    parser.add_argument("--data_dir", type=str, default="/content/drive/MyDrive/saudi/saudi-signfor-all-competition",
                      help="Directory containing H5/CSV/JSON data files (Colab Path)")
    parser.add_argument("--output_dir", type=str, default="/content/drive/MyDrive/saudi/output",
                      help="Directory to save trained models and logs (Colab Path)")
    parser.add_argument("--results_dir", type=str, default="/content/drive/MyDrive/saudi/results",
                      help="Directory to save evaluation results and predictions (Colab Path)")

    # File names exactly as provided
    parser.add_argument("--train_h5_file", type=str, default="SSL.keypoints.train_signers_train_sentences.0.h5",
                       help="Filename of training H5 file (within data_dir)")
    parser.add_argument("--train_csv_file", type=str, default="SSL.keypoints.train_signers_train_sentences.csv",
                       help="Filename of training CSV file (within data_dir)")
    parser.add_argument("--test_h5_file", type=str, default="SSL.keypoints.test_signers_test_sentences.h5",
                       help="Filename of test H5 file (within data_dir)")
    parser.add_argument("--test_json_file", type=str, default="SSL.keypoints.test_signers_test_sentences.json",
                       help="Filename of test JSON file (within data_dir)")

    parser.add_argument("--seed", type=int, default=42, help="Random seed for reproducibility")
    parser.add_argument("--mbart_model_name", type=str, default="facebook/mbart-large-50-many-to-many-mmt", help="Pre-trained mBART model name")
    parser.add_argument("--target_lang", type=str, default="ar_AR", help="mBART language code for Arabic")
    parser.add_argument("--feature_dim", type=int, default=208, help="Keypoint feature dimension (check H5 files)")
    parser.add_argument("--epochs", type=int, default=15, help="Number of training epochs")
    parser.add_argument("--warmup_steps", type=int, default=500, help="Number of warmup steps for learning rate scheduler")
    parser.add_argument("--weight_decay", type=float, default=0.01, help="Weight decay for optimizer")
    parser.add_argument("--logging_steps", type=int, default=100, help="Log training info every N steps")
    parser.add_argument("--save_total_limit", type=int, default=2, help="Maximum number of checkpoints to save")
    parser.add_argument("--fp16", action="store_true", default=True, help="Enable FP16 training")

    # A100 GPU optimizations
    parser.add_argument("--gradient_checkpointing", action="store_true", default=False, help="Enable gradient checkpointing")
    parser.add_argument("--max_seq_length", type=int, default=512, help="Max sequence length for keypoints")
    parser.add_argument("--max_label_length", type=int, default=128, help="Max sequence length for target Arabic text")
    parser.add_argument("--train_batch_size", type=int, default=4, help="Training batch size per device")
    parser.add_argument("--eval_batch_size", type=int, default=16, help="Evaluation batch size per device")
    parser.add_argument("--gradient_accumulation_steps", type=int, default=1, help="Number of steps to accumulate gradients")
    parser.add_argument("--num_beams", type=int, default=5, help="Number of beams for generation during evaluation")

    # Use parse_known_args to ignore extra arguments from environments like Jupyter
    args, unknown = parser.parse_known_args()
    if unknown:
        print(f"Ignoring unrecognized arguments: {unknown}")

    # Calculate effective batch size
    num_devices = torch.cuda.device_count() if torch.cuda.is_available() else 1
    effective_batch_size = args.train_batch_size * args.gradient_accumulation_steps * num_devices
    print(f"Effective batch size: {effective_batch_size} (train_bs={args.train_batch_size} * grad_accum={args.gradient_accumulation_steps} * devices={num_devices})")

    # Create directories if they don't exist
    os.makedirs(args.data_dir, exist_ok=True)
    os.makedirs(args.output_dir, exist_ok=True)
    os.makedirs(args.results_dir, exist_ok=True)

    return args


In [ ]:



# --- Utility Functions ---
def set_seed(seed_value):
    """Set random seed for reproducibility across libraries."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    transformers_set_seed(seed_value)

def check_and_prompt_for_file(file_path, file_type):
    """Check if a file exists and prompt for an alternative path if not found."""
    if os.path.exists(file_path):
        print(f"Found {file_type} file: {file_path}")
        return file_path

    print(f"ERROR: {file_type} file not found at {file_path}")

    user_input = None
    while True:
        user_input = input(f"Please enter an alternative path for the {file_type} file or type 'skip' to bypass: ")

        if user_input.lower() == 'skip':
            print(f"Skipping {file_type} file")
            return None

        if os.path.exists(user_input):
            print(f"Found {file_type} file at: {user_input}")
            return user_input
        else:
            print(f"File not found at: {user_input}")

In [ ]:
def preload_h5_info(file_path):
    """
    Scans an H5 file and extracts key information:
    - List of valid keys
    - Mapping of keys to sequence lengths
    - Feature dimension
    """
    keys = []
    lengths = {}
    feature_dim = None
    if not file_path or not os.path.exists(file_path):
        print(f"Error: H5 path {file_path} invalid or not found.")
        return keys, lengths, feature_dim
    try:
        with h5py.File(file_path, "r") as f:
            print(f"Scanning HDF5 {file_path}...")
            valid_keys = list(f.keys())
            for key in tqdm(valid_keys, desc="Preloading HDF5 info"):
                try:
                    item = f[key]
                    seq_len = 0
                    current_dim = 0

                    # Handle potential group structure
                    if isinstance(item, h5py.Group):
                        data_keys = list(item.keys())
                        if data_keys:
                            data = item[data_keys[0]]
                            if isinstance(data, h5py.Dataset):
                                seq_len = data.shape[0]
                                if len(data.shape) > 1:
                                    current_dim = data.shape[1]
                        else:
                             continue # Skip empty group
                    elif isinstance(item, h5py.Dataset):
                        seq_len = item.shape[0]
                        if len(item.shape) > 1:
                            current_dim = item.shape[1]
                    else:
                        continue # Skip if not a group or dataset

                    if seq_len > 0:
                        keys.append(key)
                        lengths[key] = seq_len
                        if feature_dim is None and current_dim > 0:
                            feature_dim = current_dim
                        elif feature_dim is not None and current_dim != feature_dim and current_dim > 0:
                             pass

                except Exception as e:
                    print(f"Error processing key {key}: {e}")

            print(f"Found {len(keys)} non-empty keys in HDF5.")
            if feature_dim is not None:
                print(f"Detected primary feature dimension: {feature_dim}")
            else:
                print("Warning: Could not detect feature dimension from H5 file.")

            return keys, lengths, feature_dim
    except Exception as e:
        print(f"Error opening HDF5 {file_path}: {e}")
        return keys, lengths, feature_dim


In [ ]:
def extract_signer_sentence_ids(key):
    """
    Extract signer ID and sentence ID from the key format (e.g., '00_0001').

    Args:
        key (str): The key in format 'XX_YYYY'

    Returns:
        tuple: (signer_id, sentence_id) or (None, None) if format doesn't match
    """
    try:
        if '_' in key:
            parts = key.split('_')
            if len(parts) == 2:
                signer_id = parts[0]
                sentence_id = parts[1]
                return signer_id, sentence_id
    except Exception:
        pass
    return None, None


In [ ]:

def create_validation_splits_with_strict_separation(train_h5_keys, id_to_sentence_map, train_h5_lengths, max_seq_length, seed=42):
    """
    Create validation splits with STRICT separation from training data.
    This ensures validation data is NEVER used in training.

    Args:
        train_h5_keys: List of keys from training H5 file
        id_to_sentence_map: Mapping from keys to sentence translations
        train_h5_lengths: Mapping from keys to sequence lengths
        max_seq_length: Maximum sequence length to include
        seed: Random seed for reproducibility

    Returns:
        dict: Dictionary of validation splits and metadata
    """
    print("\n--- Creating Validation Splits with STRICT Separation ---")

    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)

    # Filter keys by length and valid translations
    valid_keys = [key for key in train_h5_keys
                  if key in id_to_sentence_map
                  and id_to_sentence_map[key]
                  and isinstance(id_to_sentence_map[key], str)
                  and id_to_sentence_map[key].strip()
                  and key in train_h5_lengths
                  and train_h5_lengths[key] <= max_seq_length]

    print(f"Valid keys after filtering: {len(valid_keys)}")

    # Extract signer and sentence information
    signer_to_keys = {}
    sentence_to_keys = {}
    key_info = {}

    for key in valid_keys:
        signer_id, sentence_id = extract_signer_sentence_ids(key)
        sentence = id_to_sentence_map[key]

        if signer_id is not None:
            if signer_id not in signer_to_keys:
                signer_to_keys[signer_id] = []
            signer_to_keys[signer_id].append(key)

        if sentence not in sentence_to_keys:
            sentence_to_keys[sentence] = []
        sentence_to_keys[sentence].append(key)

        key_info[key] = {
            'signer_id': signer_id,
            'sentence_id': sentence_id,
            'sentence': sentence
        }

    unique_signers = list(signer_to_keys.keys())
    unique_sentences = list(sentence_to_keys.keys())

    print(f"Found {len(unique_signers)} unique signers and {len(unique_sentences)} unique sentences")

    # Create validation splits with NO OVERLAP with training
    splits = {}

    # Split 1: Standard hold-out (completely random keys)
    train_standard, val_standard = train_test_split(
        valid_keys, test_size=0.1, random_state=seed)

    # Ensure strict separation
    assert set(train_standard).isdisjoint(set(val_standard)), "ERROR: Train and validation sets overlap!"

    splits['standard'] = {
        'name': 'Standard Random Split',
        'description': '10% of data randomly selected, completely held out from training',
        'train_keys': train_standard,
        'val_keys': val_standard,
        'challenge_level': 1
    }

    print(f"Standard split: {len(train_standard)} train, {len(val_standard)} validation samples")
    print(f"Verified: Train and validation sets have NO overlap")

    # Split 2: Unseen sentences
    if len(unique_sentences) >= 10:
        # Hold out 10% of sentences completely
        val_sentence_count = max(int(len(unique_sentences) * 0.1), 5)  # At least 5 sentences
        val_sentences = set(random.sample(unique_sentences, k=val_sentence_count))

        # Ensure sentences in validation are NEVER in training
        val_unseen_sentence_keys = []
        train_seen_sentence_keys = []

        for key in valid_keys:
            sentence = id_to_sentence_map[key]
            if sentence in val_sentences:
                val_unseen_sentence_keys.append(key)
            else:
                train_seen_sentence_keys.append(key)

        # Verify strict separation
        assert set(train_seen_sentence_keys).isdisjoint(set(val_unseen_sentence_keys)), "ERROR: Train and unseen sentence validation sets overlap!"

        splits['unseen_sentences'] = {
            'name': 'Unseen Sentences Split',
            'description': f'Hold out {len(val_sentences)} complete sentences (never seen during training)',
            'train_keys': train_seen_sentence_keys,
            'val_keys': val_unseen_sentence_keys,
            'challenge_level': 2,
            'val_sentences': val_sentences  # Store these for reference
        }

        print(f"Unseen sentences split: {len(train_seen_sentence_keys)} train, {len(val_unseen_sentence_keys)} validation samples")
        print(f"Verified: Train and validation sets have NO overlap")

    # Split 3: Unseen signers
    if len(unique_signers) >= 5:
        # Hold out 20% of signers completely
        val_signer_count = max(int(len(unique_signers) * 0.2), 2)  # At least 2 signers
        val_signers = set(random.sample(unique_signers, k=val_signer_count))

        # Ensure signers in validation are NEVER in training
        val_unseen_signer_keys = []
        train_seen_signer_keys = []

        for key in valid_keys:
            signer_id, _ = extract_signer_sentence_ids(key)
            if signer_id in val_signers:
                val_unseen_signer_keys.append(key)
            else:
                train_seen_signer_keys.append(key)

        # Verify strict separation
        assert set(train_seen_signer_keys).isdisjoint(set(val_unseen_signer_keys)), "ERROR: Train and unseen signer validation sets overlap!"

        splits['unseen_signers'] = {
            'name': 'Unseen Signers Split',
            'description': f'Hold out {len(val_signers)} complete signers (never seen during training)',
            'train_keys': train_seen_signer_keys,
            'val_keys': val_unseen_signer_keys,
            'challenge_level': 3,
            'val_signers': val_signers  # Store these for reference
        }

        print(f"Unseen signers split: {len(train_seen_signer_keys)} train, {len(val_unseen_signer_keys)} validation samples")
        print(f"Verified: Train and validation sets have NO overlap")

    # Split 4: Test-like challenge (both unseen signers and sentences)
    if 'unseen_signers' in splits and 'unseen_sentences' in splits:
        # Extract a different set of signers and sentences
        # We need to ensure we don't reuse the ones already held out
        used_val_signers = set(splits['unseen_signers']['val_signers'])
        used_val_sentences = set(splits['unseen_sentences']['val_sentences'])

        remaining_signers = [s for s in unique_signers if s not in used_val_signers]
        remaining_sentences = [s for s in unique_sentences if s not in used_val_sentences]

        if len(remaining_signers) >= 2 and len(remaining_sentences) >= 5:
            challenge_signer_count = min(val_signer_count, len(remaining_signers))
            challenge_sentence_count = min(val_sentence_count, len(remaining_sentences))

            challenge_signers = set(random.sample(remaining_signers, k=challenge_signer_count))
            challenge_sentences = set(random.sample(remaining_sentences, k=challenge_sentence_count))

            # Find keys that have EITHER challenge signers OR challenge sentences
            # This creates a strict train/val separation while ensuring enough validation samples
            val_challenge_keys = []
            train_challenge_keys = []

            for key in valid_keys:
                signer_id, _ = extract_signer_sentence_ids(key)
                sentence = id_to_sentence_map[key]

                # If this key has either a challenge signer or sentence, it goes to validation
                if (signer_id in challenge_signers) or (sentence in challenge_sentences):
                    val_challenge_keys.append(key)
                else:
                    train_challenge_keys.append(key)

            # Verify strict separation
            assert set(train_challenge_keys).isdisjoint(set(val_challenge_keys)), "ERROR: Train and challenge validation sets overlap!"

            # Only create this split if we have enough validation samples
            if len(val_challenge_keys) >= 10:
                splits['challenge'] = {
                    'name': 'Test-Like Challenge Split',
                    'description': 'Both unseen signers and unseen sentences (strictly held out from training)',
                    'train_keys': train_challenge_keys,
                    'val_keys': val_challenge_keys,
                    'challenge_level': 4,
                    'val_signers': challenge_signers,
                    'val_sentences': challenge_sentences
                }

                print(f"Challenge split: {len(train_challenge_keys)} train, {len(val_challenge_keys)} validation samples")
                print(f"Verified: Train and validation sets have NO overlap")

    return splits, key_info




In [ ]:
def verify_split_separation(splits):
    """
    Verify that each split maintains strict separation between its own training and validation data.

    Args:
        splits: Dictionary of validation splits

    Returns:
        bool: True if all individual splits have strict separation, False otherwise
    """
    print("\n--- Verifying Individual Split Separation ---")
    all_valid = True

    # Check each split for strict separation between its own train and validation sets
    for name, split_info in splits.items():
        train_keys = set(split_info['train_keys'])
        val_keys = set(split_info['val_keys'])

        # Verify no overlap between train and val within this split
        if not train_keys.isdisjoint(val_keys):
            overlap = len(train_keys.intersection(val_keys))
            print(f"ERROR: Split '{name}' has {overlap} overlapping keys between train and validation!")
            all_valid = False
        else:
            print(f"✓ Split '{name}' has strict separation between train and validation sets")

    # Now provide info about cross-split overlap (not an error, just informational)
    print("\n--- Cross-Split Overlap Information (Expected) ---")
    split_names = list(splits.keys())
    for i, name1 in enumerate(split_names):
        for j in range(i+1, len(split_names)):
            name2 = split_names[j]
            train_keys1 = set(splits[name1]['train_keys'])
            val_keys1 = set(splits[name1]['val_keys'])
            train_keys2 = set(splits[name2]['train_keys'])
            val_keys2 = set(splits[name2]['val_keys'])

            train_train_overlap = len(train_keys1.intersection(train_keys2))
            val_val_overlap = len(val_keys1.intersection(val_keys2))

            # Show overlap info for training and validation sets
            print(f"Info: Splits '{name1}' and '{name2}':")
            print(f"  - {train_train_overlap} overlapping training keys")
            print(f"  - {val_val_overlap} overlapping validation keys")

            # Also show cross-overlap (informational only)
            train1_val2_overlap = len(train_keys1.intersection(val_keys2))
            train2_val1_overlap = len(train_keys2.intersection(val_keys1))

            print(f"  - {train1_val2_overlap} keys in both '{name1}' train and '{name2}' validation")
            print(f"  - {train2_val1_overlap} keys in both '{name2}' train and '{name1}' validation")

    if all_valid:
        print("\n✓ All individual splits maintain strict separation between their own train and validation sets!")
        print("NOTE: The overlap between different splits is normal and expected.")
        print("The training will use only ONE split (the most challenging one) for both training and validation.")
    else:
        print("\n✗ One or more splits have internal train/validation overlap!")

    return all_valid

In [ ]:
def load_and_prepare_test_data(test_h5_path, test_json_path, test_h5_keys=None, test_h5_lengths=None):
    """
    Load and prepare the official test dataset

    Args:
        test_h5_path: Path to test H5 file
        test_json_path: Path to test JSON file
        test_h5_keys: List of keys from test H5 file (if already loaded)
        test_h5_lengths: Mapping from keys to sequence lengths (if already loaded)

    Returns:
        list: List of valid test keys
    """
    test_ids_from_json = []

    # Load test IDs from JSON
    if test_json_path and os.path.exists(test_json_path):
        try:
            with open(test_json_path, "r") as f:
                test_data_json = json.load(f)
                if isinstance(test_data_json, list) and test_data_json and isinstance(test_data_json[0], dict) and "id" in test_data_json[0]:
                    test_ids_from_json = [item["id"] for item in test_data_json]
                    print(f"Loaded {len(test_ids_from_json)} test IDs from {test_json_path}")
                else:
                    print(f"Warning: Test JSON format not recognized or empty at {test_json_path}.")
        except Exception as e:
            print(f"Error reading test JSON {test_json_path}: {e}")

    # If we don't have keys/lengths yet, load them from H5
    if test_h5_keys is None or test_h5_lengths is None:
        if test_h5_path and os.path.exists(test_h5_path):
            test_h5_keys, test_h5_lengths, _ = preload_h5_info(test_h5_path)
        else:
            print(f"Warning: Test H5 file not found at {test_h5_path}")
            return []

    # Find keys that exist in both H5 and JSON (if JSON was loaded)
    valid_test_keys = []
    if test_ids_from_json:
        valid_test_keys = [key for key in test_h5_keys if key in test_ids_from_json and key in test_h5_lengths]
    else:
        valid_test_keys = [key for key in test_h5_keys if key in test_h5_lengths]

    print(f"Found {len(valid_test_keys)} valid test keys for evaluation")
    return valid_test_keys

In [ ]:
def create_datasets_from_splits(splits, train_h5_path, id_to_sentence_map, train_h5_lengths,
                               tokenizer, max_seq_length, max_label_length, feature_dim, target_lang,
                               test_h5_path=None, test_h5_lengths=None):
    """
    Create training and validation datasets from the splits.

    Args:
        splits: Dictionary of validation splits
        train_h5_path: Path to training H5 file
        id_to_sentence_map: Mapping from keys to sentence translations
        train_h5_lengths: Mapping from keys to sequence lengths
        tokenizer: The tokenizer
        max_seq_length: Maximum sequence length
        max_label_length: Maximum label length
        feature_dim: Feature dimension
        target_lang: Target language
        test_h5_path: Path to test H5 file
        test_h5_lengths: Mapping from test keys to sequence lengths

    Returns:
        dict: Dictionary of datasets for each split
    """
    print("\n--- Creating Datasets from Splits ---")

    datasets = {}

    for split_name, split_info in splits.items():
        print(f"\nCreating datasets for '{split_info['name']}' split:")

        train_keys = split_info['train_keys']
        val_keys = split_info['val_keys']

        # Determine if this is using test data
        is_test_split = 'test' in split_name and test_h5_path is not None
        h5_path = test_h5_path if is_test_split else train_h5_path
        h5_lengths = test_h5_lengths if is_test_split else train_h5_lengths

        # For test validation, we don't have translations
        empty_map = {}

        # Create training dataset
        train_dataset = None
        if not is_test_split and train_keys:
            try:
                train_dataset = SignDataset(
                    h5_file_path=h5_path,
                    data_keys_to_use=train_keys,
                    id_to_sentence_map=id_to_sentence_map,
                    tokenizer=tokenizer,
                    h5_lengths_map=h5_lengths,
                    max_seq_length=max_seq_length,
                    max_label_length=max_label_length,
                    feature_dim=feature_dim,
                    target_lang=target_lang,
                    is_test_set=False
                )
                print(f"  Created training dataset with {len(train_dataset)} samples")
            except Exception as e:
                print(f"  Error creating training dataset: {e}")

        # Create validation dataset
        val_dataset = None
        if val_keys:
            try:
                val_dataset = SignDataset(
                    h5_file_path=h5_path,
                    data_keys_to_use=val_keys,
                    # Use empty map for test validation to mark as test data
                    id_to_sentence_map=empty_map if is_test_split else id_to_sentence_map,
                    tokenizer=tokenizer,
                    h5_lengths_map=h5_lengths,
                    max_seq_length=max_seq_length,
                    max_label_length=max_label_length,
                    feature_dim=feature_dim,
                    target_lang=target_lang,
                    is_test_set=is_test_split
                )
                print(f"  Created validation dataset with {len(val_dataset)} samples")
            except Exception as e:
                print(f"  Error creating validation dataset: {e}")

        datasets[split_name] = {
            'train': train_dataset,
            'validation': val_dataset,
            'info': split_info
        }

    return datasets

# --- Custom Data Collator --- #
@dataclass
class CustomDataCollator:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None # Not used for features here, padding done in dataset
    pad_to_multiple_of: Optional[int] = None # Applied to labels
    label_pad_token_id: int = -100
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        input_features = [f["input_features"] for f in features]
        attention_mask = [f["attention_mask"] for f in features]
        labels = [f["labels"] for f in features] # These are the ones to pad dynamically
        decoder_attention_mask = [f["decoder_attention_mask"] for f in features]

        batch_input_features = torch.stack(input_features)
        batch_attention_mask = torch.stack(attention_mask)

        max_label_len = max(len(lab) for lab in labels)
        if self.pad_to_multiple_of is not None:
            max_label_len = (
                (max_label_len + self.pad_to_multiple_of - 1)
                // self.pad_to_multiple_of
                * self.pad_to_multiple_of
            )

        padded_labels = []
        padded_decoder_attention_mask = []
        for lab, dec_attn in zip(labels, decoder_attention_mask):
            remainder = max_label_len - len(lab)
            padded_labels.append(F.pad(lab, (0, remainder), value=self.label_pad_token_id))
            padded_decoder_attention_mask.append(F.pad(dec_attn, (0, remainder), value=0))

        batch_labels = torch.stack(padded_labels)
        batch_decoder_attention_mask = torch.stack(padded_decoder_attention_mask)

        batch = {
            "input_features": batch_input_features,
            "attention_mask": batch_attention_mask,
            "labels": batch_labels,
            "decoder_attention_mask": batch_decoder_attention_mask,
        }
        return batch

# --- Model Definition --- #
class VisualHead(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.1):
        super().__init__()
        self.projection = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.GELU(),
            nn.Dropout(dropout_rate)
        )
        print(f"Initialized VisualHead: Linear({input_dim} -> {output_dim})")

    def forward(self, features):
        # The issue is here - it should use self.projection, not self.visual_head
        projected_features = self.projection(features)
        return projected_features

class SignTranslationModel(nn.Module):
    def __init__(self, feature_dim, mbert_model_name, dropout_rate=0.1):
        super().__init__()
        print(f"Initializing SignTranslationModel:")
        print(f"  Loading Base mBART Model From: {mbert_model_name}")
        print(f"  Feature Dim (Input): {feature_dim}")

        try:
            # CRUCIAL CHANGE: Set config before loading model
            from transformers import MBartConfig
            config = MBartConfig.from_pretrained(mbert_model_name)

            # Disable SDPA in config before loading
            config._attn_implementation = "eager"

            # Now load with our config
            self.mbart = MBartForConditionalGeneration.from_pretrained(
                mbert_model_name,
                config=config
            )
            print(f"  Base mBART model loaded successfully with eager attention.")
        except Exception as e:
            print(f"ERROR loading base mBART model from {mbert_model_name}: {e}")
            raise

        self.model_hidden_dim = self.mbart.config.d_model
        print(f"  Model Hidden Dim (d_model): {self.model_hidden_dim}")

        self.visual_head = VisualHead(feature_dim, self.model_hidden_dim, dropout_rate)

        # Initialize generation_config
        self.generation_config = None  # Will be set later in training setup

    def forward(self, input_features, attention_mask, labels=None, decoder_attention_mask=None, **kwargs):
        input_features = input_features.float()
        try:
            inputs_embeds = self.visual_head(input_features)
        except Exception as e:
            print(f"VisualHead projection error: {e}\nInput shape: {input_features.shape}, Input dtype: {input_features.dtype}")
            raise

        try:
            outputs = self.mbart(
                inputs_embeds=inputs_embeds,
                attention_mask=attention_mask,
                labels=labels,
                decoder_attention_mask=decoder_attention_mask,
                return_dict=True
            )
        except Exception as e:
            print(f"mBART forward error: {e}")
            print(f"  inputs_embeds shape: {inputs_embeds.shape if inputs_embeds is not None else 'None'}, dtype: {inputs_embeds.dtype if inputs_embeds is not None else 'None'}")
            print(f"  attention_mask shape: {attention_mask.shape if attention_mask is not None else 'None'}, dtype: {attention_mask.dtype if attention_mask is not None else 'None'}")
            print(f"  labels shape: {labels.shape if labels is not None else 'None'}, dtype: {labels.dtype if labels is not None else 'None'}")
            print(f"  decoder_attention_mask shape: {decoder_attention_mask.shape if decoder_attention_mask is not None else 'None'}, dtype: {decoder_attention_mask.dtype if decoder_attention_mask is not None else 'None'}")
            raise
        return outputs

    @torch.no_grad()
    def generate(self, input_features, attention_mask, **generate_kwargs):
        self.eval()
        input_features = input_features.float() # Ensure float input
        inputs_embeds = self.visual_head(input_features)

        # ATTENTION MASK FIX - Ensure correct shape for generate
        # During beam search, the attention mask shape can cause issues
        # We need to ensure it's correctly shaped for the mBART model

        if attention_mask is not None:
            # Get batch size
            batch_size = attention_mask.size(0)

            # Ensure attention mask is the correct shape by reshaping
            attention_mask = attention_mask.view(batch_size, -1)

            # Debug log the shape
            # print(f"Generate: Attention mask reshaped to {attention_mask.shape}")

        decoder_start_token_id = self.mbart.config.forced_bos_token_id # Default
        if "tgt_lang" in generate_kwargs:
            try:
                lang_code = generate_kwargs["tgt_lang"]
                lang_token_id = global_tokenizer.lang_code_to_id.get(lang_code)
                if lang_token_id is not None:
                   decoder_start_token_id = lang_token_id
                else:
                    print(f"Warning: Target language '{lang_code}' not found in tokenizer lang_code_to_id. Using default BOS: {decoder_start_token_id}.")
            except Exception as e:
                 print(f"Warning: Error accessing tokenizer for target language '{generate_kwargs['tgt_lang']}': {e}. Using default BOS: {decoder_start_token_id}")

        # Use the model's generation_config if available, otherwise use defaults
        if hasattr(self, 'generation_config') and self.generation_config is not None:
            # Clone and update the generation config
            from copy import deepcopy
            gen_config = deepcopy(self.generation_config)

            # Update with any kwargs provided
            for k, v in generate_kwargs.items():
                if hasattr(gen_config, k):
                    setattr(gen_config, k, v)

            # Extra safety check for critical parameters
            if "max_length" in generate_kwargs:
                gen_config.max_length = generate_kwargs["max_length"]
            if "num_beams" in generate_kwargs:
                gen_config.num_beams = generate_kwargs["num_beams"]
            if "decoder_start_token_id" in generate_kwargs:
                gen_config.decoder_start_token_id = generate_kwargs["decoder_start_token_id"]
            else:
                gen_config.decoder_start_token_id = decoder_start_token_id

            # Run generation with the config object
            try:
                return self.mbart.generate(
                    inputs_embeds=inputs_embeds,
                    attention_mask=attention_mask,
                    generation_config=gen_config
                )
            except Exception as e:
                print(f"Generate failed with generation_config: {e}")
                print("Falling back to kwargs method...")

        # Fall back to kwargs method
        final_generate_kwargs = {
            "max_length": generate_kwargs.get("max_length", self.mbart.config.max_length),
            "num_beams": generate_kwargs.get("num_beams", self.mbart.config.num_beams),
            "early_stopping": generate_kwargs.get("early_stopping", True),
            "decoder_start_token_id": decoder_start_token_id,
        }
        for k, v in generate_kwargs.items():
            if k not in ["tgt_lang", "max_length", "num_beams", "early_stopping", "decoder_start_token_id", "forced_decoder_ids"]:
                 final_generate_kwargs[k] = v

        return self.mbart.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            **final_generate_kwargs
        )

    def save_pretrained(self, save_dir):
        """Save model to the specified directory."""
        os.makedirs(save_dir, exist_ok=True)

        # Save mBART config
        self.mbart.config.save_pretrained(save_dir)

        # Save model state_dict
        torch.save(self.state_dict(), os.path.join(save_dir, "pytorch_model.bin"))

        # Save generation config if it exists
        if hasattr(self, 'generation_config') and self.generation_config is not None:
            self.generation_config.save_pretrained(save_dir)

        print(f"Model saved to {save_dir}")

    @classmethod
    def from_pretrained(cls, load_dir, device=None):
        """Load model from the specified directory."""
        # Load config
        from transformers import MBartConfig
        config = MBartConfig.from_pretrained(load_dir)

        # Determine feature_dim (needed to initialize the model)
        # This might be stored in the config or we need to infer it
        feature_dim = getattr(config, "feature_dim", 208)  # Default to 208 if not found

        # Create new model instance
        model = cls(
            feature_dim=feature_dim,
            mbert_model_name=load_dir
        )

        # Load state_dict
        state_dict_path = os.path.join(load_dir, "pytorch_model.bin")
        if os.path.exists(state_dict_path):
            state_dict = torch.load(state_dict_path, map_location="cpu")
            model.load_state_dict(state_dict)
            print(f"Model weights loaded from {state_dict_path}")
        else:
            print(f"Warning: No model weights found at {state_dict_path}")

        # Load generation config if it exists
        generation_config_path = os.path.join(load_dir, "generation_config.json")
        if os.path.exists(generation_config_path):
            model.generation_config = GenerationConfig.from_pretrained(load_dir)
            print(f"Generation config loaded from {generation_config_path}")

        # Move to device if specified
        if device is not None:
            model = model.to(device)

        return model

    def gradient_checkpointing_enable(self, gradient_checkpointing_kwargs=None):
        """Enable gradient checkpointing for memory efficiency."""
        print("Enabling gradient checkpointing on underlying mBART model...")
        if hasattr(self.mbart, "gradient_checkpointing_enable"):
            self.mbart.gradient_checkpointing_enable(gradient_checkpointing_kwargs=gradient_checkpointing_kwargs)
            print("Gradient checkpointing enabled on mBART.")
        else:
            print("Warning: Underlying mBART model does not have 'gradient_checkpointing_enable' method.")

# --- Evaluation Metrics --- #
bleu_metric = evaluate.load("sacrebleu")
global_tokenizer = None # To be set after tokenizer initialization

def postprocess_text(preds, labels):
    """Process predictions and labels for evaluation."""
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    """Compute BLEU score for evaluation."""
    global global_tokenizer
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0] # Predictions are often tuples

    labels = np.where(labels != -100, labels, global_tokenizer.pad_token_id)

    try:
        decoded_preds = global_tokenizer.batch_decode(preds, skip_special_tokens=True)
    except Exception as e:
        print(f"Error decoding predictions: {e}")
        decoded_preds = ["<DECODE_ERROR>"] * len(preds)

    try:
        decoded_labels = global_tokenizer.batch_decode(labels, skip_special_tokens=True)
    except Exception as e:
        print(f"Error decoding labels: {e}")
        decoded_labels = ["<DECODE_ERROR>"] * len(labels)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    try:
        if not isinstance(decoded_preds, list) or not all(isinstance(p, str) for p in decoded_preds):
             raise ValueError("Decoded predictions are not in the expected format (List[str]).")
        if not isinstance(decoded_labels, list) or not all(isinstance(ref, list) and all(isinstance(s, str) for s in ref) for ref in decoded_labels):
            raise ValueError("Decoded labels are not in the expected format (List[List[str]]).")

        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}
    except Exception as e:
        print(f"Error computing BLEU: {e}")
        result = {"bleu": 0.0} # Default to 0 on error

    prediction_lens = [np.count_nonzero(pred != global_tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    result = {k: round(v, 4) for k, v in result.items()}
    return result

def train_and_evaluate_with_strict_separation(model, datasets, args, tokenizer, data_collator):
    """
    Train and evaluate the model with strict separation between training and validation.

    Args:
        model: The model to train
        datasets: Dictionary of datasets for each split
        args: Training arguments
        tokenizer: The tokenizer
        data_collator: The data collator

    Returns:
        The trained model
    """
    print("\n=== Training and Evaluation with Strict Separation ===")

    # Create results directory
    os.makedirs(args.results_dir, exist_ok=True)
    results_file = os.path.join(args.results_dir, "training_results.txt")

    # Select primary split for training (highest challenge level with valid datasets)
    available_splits = [(name, info['info']['challenge_level'])
                       for name, info in datasets.items()
                       if info['train'] is not None and info['validation'] is not None]

    if not available_splits:
        print("ERROR: No valid splits found for training. Aborting.")
        return model

    # Sort by challenge level (descending)
    available_splits.sort(key=lambda x: x[1], reverse=True)
    primary_split = available_splits[0][0]

    primary_train = datasets[primary_split]['train']
    primary_validation = datasets[primary_split]['validation']
    split_info = datasets[primary_split]['info']

    print(f"\nUsing '{split_info['name']}' as primary split for training.")
    print(f"Challenge level: {split_info['challenge_level']}")
    print(f"Description: {split_info['description']}")
    print(f"Training samples: {len(primary_train)}")
    print(f"Validation samples: {len(primary_validation)}")

    # Setup logging
    with open(results_file, "w") as f:
        f.write("=== Sign Language Translation Training Results ===\n\n")
        f.write(f"Model: mBART with Visual Head\n")
        f.write(f"Primary split: {split_info['name']}\n")
        f.write(f"Challenge level: {split_info['challenge_level']}\n")
        f.write(f"Description: {split_info['description']}\n")
        f.write(f"Training samples: {len(primary_train)}\n")
        f.write(f"Validation samples: {len(primary_validation)}\n\n")

        f.write("Other available splits:\n")
        for name, info in datasets.items():
            if name != primary_split and info['train'] is not None and info['validation'] is not None:
                f.write(f"- {info['info']['name']}: {len(info['train'])} train, {len(info['validation'])} validation\n")
                f.write(f"  {info['info']['description']}\n")

        f.write("\n=== TRAINING PROGRESS ===\n")

    # Setup target language tokens
    lang_token_id = tokenizer.lang_code_to_id.get(args.target_lang)
    if lang_token_id is None:
        print(f"Warning: Target language code '{args.target_lang}' not found in tokenizer's lang_code_to_id map.")
        lang_token_id = model.mbart.config.decoder_start_token_id
        print(f"Using fallback decoder_start_token_id: {lang_token_id}")

    # Generation config
    generation_config = GenerationConfig(
        max_length=args.max_label_length,
        num_beams=args.num_beams,
        early_stopping=True,
        decoder_start_token_id=lang_token_id,
        forced_bos_token_id=lang_token_id,
    )

    model.generation_config = generation_config

    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=args.output_dir,
        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.train_batch_size,
        per_device_eval_batch_size=args.eval_batch_size,
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        warmup_steps=args.warmup_steps,
        weight_decay=args.weight_decay,
        logging_dir=os.path.join(args.output_dir, "logs"),
        logging_steps=args.logging_steps,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=args.save_total_limit,
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        greater_is_better=True,
        predict_with_generate=True,
        generation_max_length=args.max_label_length,
        generation_num_beams=args.num_beams,
        fp16=args.fp16 and torch.cuda.is_available(),
        gradient_checkpointing=args.gradient_checkpointing,
        report_to=["tensorboard"],
        dataloader_num_workers=2,
        dataloader_pin_memory=True if torch.cuda.is_available() else False,
        save_safetensors=False,
    )

    # Initialize trainer with primary validation dataset
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=primary_train,
        eval_dataset=primary_validation,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # Early stopping setup
    early_stopping_patience = 3
    best_epoch = 0
    best_bleu = 0
    no_improvement_count = 0

    # Train the model
    print("\n=== Starting Training ===")

    # Check for existing checkpoints
    checkpoint_dir = args.output_dir
    resume_from_checkpoint = False

    # Look for checkpoint directories
    if os.path.exists(checkpoint_dir):
        checkpoint_dirs = [d for d in os.listdir(checkpoint_dir)
                          if os.path.isdir(os.path.join(checkpoint_dir, d))
                          and d.startswith("checkpoint-")]

        if checkpoint_dirs:
            # Sort checkpoints by step number
            checkpoint_dirs.sort(key=lambda x: int(x.split("-")[1]))
            latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_dirs[-1])
            print(f"\nFound existing checkpoint: {latest_checkpoint}")
            resume_choice = input("Do you want to resume training from this checkpoint? (y/n): ").lower()

            if resume_choice == 'y':
                resume_from_checkpoint = latest_checkpoint
                print(f"Will resume training from: {resume_from_checkpoint}")
            else:
                print("Starting training from scratch.")
        else:
            print("No checkpoints found. Starting training from scratch.")

    for epoch in range(args.epochs):
        print(f"\n--- Epoch {epoch + 1}/{args.epochs} ---")

        # Train for one epoch - modified to support checkpoint resumption
        train_result = trainer.train(resume_from_checkpoint=resume_from_checkpoint)

        # Only use checkpoint for first epoch, then set to False
        if resume_from_checkpoint:
            resume_from_checkpoint = False

        train_metrics = train_result.metrics

        print(f"Training loss: {train_metrics['train_loss']:.4f}")

        # Evaluate on all validation splits
        with open(results_file, "a") as f:
            f.write(f"\nEpoch {epoch + 1}\n")
            f.write(f"Training loss: {train_metrics['train_loss']:.4f}\n")

        # Track best model
        current_bleu = None

        # Evaluate on each split's validation set
        for split_name, split_data in datasets.items():
            val_dataset = split_data['validation']

            if val_dataset is None:
                continue

            split_desc = split_data['info']['name']
            is_test_split = 'test' in split_name

            if is_test_split:
                # For test validation, we can only generate predictions, not evaluate
                print(f"\nGenerating predictions for '{split_desc}' validation")
                predictions = trainer.predict(
                    test_dataset=val_dataset,
                    metric_key_prefix=f"val_{split_name}",
                )

                with open(results_file, "a") as f:
                    f.write(f"{split_desc}: Generated predictions only (no reference translations)\n")
            else:
                # For regular validation, evaluate with metrics
                print(f"\nEvaluating on '{split_desc}' validation")
                val_metrics = trainer.evaluate(
                    eval_dataset=val_dataset,
                    metric_key_prefix=f"val_{split_name}",
                )

                bleu_score = val_metrics[f"val_{split_name}_bleu"]
                print(f"{split_desc} BLEU: {bleu_score:.2f}")

                with open(results_file, "a") as f:
                    f.write(f"{split_desc} BLEU: {bleu_score:.2f}\n")

                # Track primary split for early stopping
                if split_name == primary_split:
                    current_bleu = bleu_score

        # Check early stopping on primary validation
        if current_bleu is not None:
            if current_bleu > best_bleu:
                best_bleu = current_bleu
                best_epoch = epoch + 1
                no_improvement_count = 0

                # Save best model
                best_model_dir = os.path.join(args.output_dir, "best_model")
                os.makedirs(best_model_dir, exist_ok=True)
                model.save_pretrained(best_model_dir)
                tokenizer.save_pretrained(best_model_dir)

                print(f"New best model saved with BLEU: {best_bleu:.2f}")
            else:
                no_improvement_count += 1
                print(f"No improvement for {no_improvement_count} epochs (best: {best_bleu:.2f} at epoch {best_epoch})")

        # Check early stopping
        if no_improvement_count >= early_stopping_patience:
            print(f"\nEarly stopping after {epoch + 1} epochs!")
            with open(results_file, "a") as f:
                f.write(f"\nEarly stopping triggered. Best model was at epoch {best_epoch} with BLEU: {best_bleu:.2f}\n")
            break

    print("\n=== Training Complete ===")
    print(f"Best model was at epoch {best_epoch} with BLEU: {best_bleu:.2f}")

    # Load best model for final evaluation
    best_model_path = os.path.join(args.output_dir, "best_model")
    if os.path.exists(best_model_path):
        print(f"\nLoading best model from epoch {best_epoch}...")
        model = model.from_pretrained(best_model_path, device=device)

    # Final evaluation on all validation splits
    print("\n=== Final Evaluation ===")

    with open(results_file, "a") as f:
        f.write("\n=== FINAL EVALUATION ===\n")

    for split_name, split_data in datasets.items():
        val_dataset = split_data['validation']

        if val_dataset is None:
            continue

        split_desc = split_data['info']['name']
        is_test_split = 'test' in split_name

        if is_test_split:
            print(f"\nFinal prediction generation for '{split_desc}'")
            predict_results = trainer.predict(
                test_dataset=val_dataset,
                metric_key_prefix=f"final_{split_name}",
            )

            with open(results_file, "a") as f:
                f.write(f"Final {split_desc}: Generated predictions only\n")
        else:
            print(f"\nFinal evaluation on '{split_desc}'")
            val_metrics = trainer.evaluate(
                eval_dataset=val_dataset,
                metric_key_prefix=f"final_{split_name}",
            )

            bleu_score = val_metrics[f"final_{split_name}_bleu"]
            print(f"Final {split_desc} BLEU: {bleu_score:.2f}")

            with open(results_file, "a") as f:
                f.write(f"Final {split_desc} BLEU: {bleu_score:.2f}\n")

    # Generate predictions for official test set if available
    if 'official_test' in datasets and datasets['official_test']['validation'] is not None:
        test_dataset = datasets['official_test']['validation']

        print("\n=== Generating Official Test Set Predictions ===")

        predict_results = trainer.predict(
            test_dataset=test_dataset,
            metric_key_prefix="test",
        )

        # Save predictions
        predictions = predict_results.predictions
        predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        cleaned_predictions = [pred.strip() for pred in decoded_predictions]

        # Create submission format
        output_prediction_file = os.path.join(args.results_dir, "test_predictions.txt")
        with open(output_prediction_file, "w", encoding="utf-8") as writer:
            writer.write("ID\tPrediction\n")
            for idx, prediction in enumerate(cleaned_predictions):
                if idx < len(test_dataset.data_info):
                    sample_key = test_dataset.data_info[idx][0]
                    writer.write(f"{sample_key}\t{prediction}\n")

        print(f"Test predictions saved to {output_prediction_file}")

        # Create JSON format for submission
        submission_file = os.path.join(args.results_dir, "submission.json")
        submission_data = []

        for idx, prediction in enumerate(cleaned_predictions):
            if idx < len(test_dataset.data_info):
                sample_key = test_dataset.data_info[idx][0]
                submission_data.append({
                    "id": sample_key,
                    "translation": prediction
                })

        with open(submission_file, "w", encoding="utf-8") as f:
            json.dump(submission_data, f, ensure_ascii=False, indent=2)

        print(f"Submission file saved to {submission_file}")

    return model


NameError: name 'dataclass' is not defined

In [ ]:


# === NEW: Main Class for Streamlined Execution ===
class SignLanguageTranslationSystem:
    def __init__(self):
        self.args = None
        self.model = None
        self.tokenizer = None
        self.train_h5_keys = []
        self.train_h5_lengths = {}
        self.test_h5_keys = []
        self.test_h5_lengths = {}
        self.id_to_sentence_map = {}
        self.feature_dim = None
        self.splits = {}
        self.datasets = {}
        self.data_collator = None
        self.mount_completed = False
        self.training_completed = False

    def mount_drive(self):
        """Mount Google Drive if in Colab environment"""
        if self.mount_completed:
            print("Drive already mounted. Skipping.")
            return

        try:
            drive.mount('/content/drive')
            print("Google Drive mounted successfully.")
            self.mount_completed = True
        except Exception as e:
            print(f"Error mounting Google Drive: {e}")
            print("Proceeding without Drive mount. Ensure data/output paths are accessible.")

    def setup(self):
        """Initialize arguments and setup paths"""
        # Parse arguments
        self.args = parse_args()
        print(f"Using device: {device}")
        set_seed(self.args.seed)
        print(f"Seed set to: {self.args.seed}")

        # Create output directories
        os.makedirs(self.args.output_dir, exist_ok=True)
        os.makedirs(self.args.results_dir, exist_ok=True)

        # Construct paths
        self.TRAIN_H5_PATH = os.path.join(self.args.data_dir, self.args.train_h5_file)
        self.TRAIN_CSV_PATH = os.path.join(self.args.data_dir, self.args.train_csv_file)
        self.TEST_H5_PATH = os.path.join(self.args.data_dir, self.args.test_h5_file)
        self.TEST_JSON_PATH = os.path.join(self.args.data_dir, self.args.test_json_file)

        # Check paths
        self.TRAIN_H5_PATH = check_and_prompt_for_file(self.TRAIN_H5_PATH, "training H5")
        self.TRAIN_CSV_PATH = check_and_prompt_for_file(self.TRAIN_CSV_PATH, "training CSV")
        self.TEST_H5_PATH = check_and_prompt_for_file(self.TEST_H5_PATH, "test H5")
        self.TEST_JSON_PATH = check_and_prompt_for_file(self.TEST_JSON_PATH, "test JSON")

    def load_data(self):
        """Load CSV and H5 file data"""
        # Load CSV data
        try:
            if self.TRAIN_CSV_PATH:
                train_df = pd.read_csv(self.TRAIN_CSV_PATH)
                print(f"Loaded training CSV: {self.TRAIN_CSV_PATH}")
                print(f"Training samples in CSV: {len(train_df)}")
            else:
                print("No CSV path provided. Training will not be possible.")
                train_df = pd.DataFrame(columns=["ID", "Translation"])
        except Exception as e:
            print(f"ERROR reading training CSV: {e}")
            print("Creating empty dataframe. Training will not be possible.")
            train_df = pd.DataFrame(columns=["ID", "Translation"])

        # Create ID to sentence map
        if "ID" in train_df.columns and "Translation" in train_df.columns:
            self.id_to_sentence_map = pd.Series(train_df.Translation.values, index=train_df.ID).to_dict()
            print(f"Created ID-to-sentence map with {len(self.id_to_sentence_map)} entries.")
        else:
            print(f"WARNING: Could not find \"ID\" and \"Translation\" columns in CSV. Please check column names.")
            if len(train_df.columns) >= 2:
                # Try to use first two columns as fallback
                column_names = train_df.columns.tolist()
                print(f"Attempting to use first two columns as fallback: {column_names[0]} and {column_names[1]}")
                self.id_to_sentence_map = pd.Series(train_df[column_names[1]].values, index=train_df[column_names[0]]).to_dict()
                print(f"Created fallback ID-to-sentence map with {len(self.id_to_sentence_map)} entries.")
            else:
                print("No suitable columns found for ID-to-sentence map.")

        # Load H5 file info
        detected_feature_dim = None

        if self.TRAIN_H5_PATH:
            self.train_h5_keys, self.train_h5_lengths, detected_feature_dim = preload_h5_info(self.TRAIN_H5_PATH)
        else:
            print("No training H5 path provided. Training will not be possible.")

        if self.TEST_H5_PATH:
            self.test_h5_keys, self.test_h5_lengths, _ = preload_h5_info(self.TEST_H5_PATH)
        else:
            print("No test H5 path provided. Evaluation will not be possible.")

        # Analyze sequence lengths
        if self.train_h5_lengths:
            self._analyze_sequence_lengths()

        # Set feature dimension
        self.feature_dim = self._determine_feature_dim(detected_feature_dim)

    def _analyze_sequence_lengths(self):
        """Analyze training sequence lengths"""
        lengths = list(self.train_h5_lengths.values())
        print("\n--- Training Sequence Length Analysis ---")
        print(f"Total sequences found in H5: {len(lengths)}")
        if lengths:
            print(f"Min length: {np.min(lengths)}")
            print(f"Max length: {np.max(lengths)}")
            print(f"Mean length: {np.mean(lengths):.2f}")
            print(f"Median length: {np.median(lengths)}")
            print(f"90th percentile: {np.percentile(lengths, 90):.2f}")
            print(f"95th percentile: {np.percentile(lengths, 95):.2f}")
            print(f"99th percentile: {np.percentile(lengths, 99):.2f}")
            num_exceeding = sum(l > self.args.max_seq_length for l in lengths)
            print(f"Num sequences > {self.args.max_seq_length}: {num_exceeding} ({num_exceeding/len(lengths)*100:.2f}%)")
            print(f"--> Based on this, {len(lengths) - num_exceeding} sequences ({100 - num_exceeding/len(lengths)*100:.2f}%) will be used with max_seq_length={self.args.max_seq_length}.")
        else:
            print("No valid sequence lengths found to analyze.")
        print("--- End Analysis ---\n")

    def _determine_feature_dim(self, detected_feature_dim):
        """Determine the feature dimension to use"""
        if not self.train_h5_keys:
            print(f"WARNING: No valid keys found in training H5 file. Training may not be possible.")
        else:
            print(f"Found {len(self.train_h5_keys)} valid keys in training H5 file.")

        final_feature_dim = self.args.feature_dim
        if detected_feature_dim is not None and detected_feature_dim != self.args.feature_dim:
            print(f"WARNING: Manually set feature_dim ({self.args.feature_dim}) differs from detected dimension ({detected_feature_dim}).")
            feature_choice = input(f"Use detected dimension {detected_feature_dim} instead of {self.args.feature_dim}? (y/n): ")
            if feature_choice.lower() == 'y':
                final_feature_dim = detected_feature_dim
                print(f"Using detected feature dimension: {final_feature_dim}")
            else:
                print(f"Using manually set feature dimension: {self.args.feature_dim}")
        elif detected_feature_dim is None:
            print(f"WARNING: Could not detect feature dimension. Using manually set value: {self.args.feature_dim}. Verify this is correct!")
        else:
            final_feature_dim = detected_feature_dim
            print(f"Using feature dimension: {final_feature_dim}")

        return final_feature_dim

    def initialize_tokenizer(self):
        """Initialize the tokenizer"""
        global global_tokenizer

        print("\nInitializing tokenizer...")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(
                self.args.mbart_model_name,
                src_lang="en_XX",
                tgt_lang=self.args.target_lang
            )
            global_tokenizer = self.tokenizer  # Set global tokenizer
            print(f"Tokenizer loaded: {self.args.mbart_model_name}")
        except Exception as e:
            print(f"Error loading tokenizer: {e}")
            print("Attempting to proceed, but this may cause issues.")

    def create_datasets(self):
        """Create validation splits and datasets"""
        # Create validation splits with strict separation
        self.splits, key_info = create_validation_splits_with_strict_separation(
            self.train_h5_keys,
            self.id_to_sentence_map,
            self.train_h5_lengths,
            self.args.max_seq_length,
            seed=self.args.seed
        )

        # Verify strict separation between all splits
        valid_splits = verify_split_separation(self.splits)

        if not valid_splits:
            print("WARNING: Split separation verification failed! Some validation data may leak into training.")
            proceed = input("Do you want to continue anyway? (y/n): ")
            if proceed.lower() != 'y':
                print("Exiting script.")
                sys.exit(1)

        # Load test data (if available)
        test_keys = load_and_prepare_test_data(
            self.TEST_H5_PATH,
            self.TEST_JSON_PATH,
            self.test_h5_keys,
            self.test_h5_lengths
        )

        # Add official test split as a separate dataset (will be test-only, no training)
        if test_keys:
            self.splits['official_test'] = {
                'name': 'Official Test Set',
                'description': 'Official test set for final submission',
                'train_keys': [],  # No training on test data
                'val_keys': test_keys,
                'challenge_level': 5  # Highest challenge level
            }

        # Create datasets from splits
        self.datasets = create_datasets_from_splits(
            splits=self.splits,
            train_h5_path=self.TRAIN_H5_PATH,
            id_to_sentence_map=self.id_to_sentence_map,
            train_h5_lengths=self.train_h5_lengths,
            tokenizer=self.tokenizer,
            max_seq_length=self.args.max_seq_length,
            max_label_length=self.args.max_label_length,
            feature_dim=self.feature_dim,
            target_lang=self.args.target_lang,
            test_h5_path=self.TEST_H5_PATH,
            test_h5_lengths=self.test_h5_lengths
        )

    def initialize_model(self):
        """Initialize the model"""
        print("\n--- Initializing Model ---")
        try:
            self.model = SignTranslationModel(
                feature_dim=self.feature_dim,
                mbert_model_name=self.args.mbart_model_name
            )

            # Apply gradient checkpointing only if explicitly enabled
            if self.args.gradient_checkpointing:
                print("Gradient Checkpointing explicitly enabled via argument.")
                self.model.gradient_checkpointing_enable()
            else:
                print("Gradient Checkpointing is DISABLED (intended for A100/faster training).")

            # Explicitly set attention implementation again after model is ready
            print("Setting attention implementation to eager on model...")
            if hasattr(self.model.mbart.config, "_attn_implementation"):
                self.model.mbart.config._attn_implementation = "eager"
            if hasattr(self.model.mbart, "_attn_implementation"):
                self.model.mbart._attn_implementation = "eager"

            # Move model to device after configuration
            self.model.to(device)
            print(f"Model instantiated and moved to {device}.")

            if hasattr(self.model, 'mbart') and hasattr(self.model.mbart, 'get_memory_footprint'):
                print(f"Model memory footprint (approx): {self.model.mbart.get_memory_footprint() / 1e9:.2f} GB")
            else:
                print("Could not retrieve model memory footprint.")
        except Exception as e:
            print(f"Error initializing model: {e}")
            print("Model initialization failed. Cannot continue with training.")
            sys.exit(1)

        # Clear cache
        print("Running GC and clearing CUDA cache before starting training...")
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print("Cache cleared.")

        # Create data collator
        self.data_collator = CustomDataCollator(tokenizer=self.tokenizer, label_pad_token_id=-100)
        print("Custom Data Collator created.")

    def train_model(self):
        """Train the model"""
        if self.training_completed:
            print("\nTraining already completed. Skipping.")
            return

        if not self.datasets:
            print("Cannot train: no valid datasets created")
            return

        # Train and evaluate with our enhanced function
        self.model = train_and_evaluate_with_strict_separation(
            model=self.model,
            datasets=self.datasets,
            args=self.args,
            tokenizer=self.tokenizer,
            data_collator=self.data_collator
        )

        self.training_completed = True
        print("\nScript execution complete!")

    def evaluate_model(self):
        """Evaluate the model on test data"""
        if not self.training_completed:
            print("Cannot evaluate: training not completed")
            return

        print("\n=== Additional Evaluation of Best Model ===")
        # Load best model
        best_model_path = os.path.join(self.args.output_dir, "best_model")
        if not os.path.exists(best_model_path):
            print(f"Best model not found at {best_model_path}")
            return

        # Custom evaluation code can be added here
        print("Model evaluation complete!")

    def generate_predictions(self):
        """Generate predictions for test data"""
        if not self.training_completed:
            print("Cannot generate predictions: training not completed")
            return

        print("\n=== Generating Test Predictions with Best Model ===")
        # Load best model
        best_model_path = os.path.join(self.args.output_dir, "best_model")
        if not os.path.exists(best_model_path):
            print(f"Best model not found at {best_model_path}")
            return

        # Generate predictions
        # Custom prediction code can be added here

        print("Test predictions generated!")



In [ ]:
# Complete Saudi Sign Language Translation Code
# For SignforAll 2025 Challenge - With Strict Validation

# Standard Libraries
import os
import sys
import evaluate
import argparse
import random
import yaml
from tqdm import tqdm
import warnings
import json
from sklearn.model_selection import train_test_split
import gc
import re
import traceback

# CRITICAL: Set environment variables BEFORE importing PyTorch
os.environ["FORCE_TORCH_SDPA_KERNEL"] = "0"
os.environ["PYTORCH_DISABLE_SDPA"] = "1"
os.environ["PYTORCH_NO_CUDA_MEMORY_EFFICIENCY_WARNING"] = "1"

# Data Handling
import numpy as np
import pandas as pd
import h5py

# Deep Learning - PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Transformers Library
from transformers import (
    AutoTokenizer,
    MBartForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    set_seed as transformers_set_seed,
    GenerationConfig
)

# Evaluation Metric
import sacrebleu

# Google Drive
try:
    from google.colab import drive
except ImportError:
    print("Not running in Google Colab environment.")

# For custom collator
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple, Union
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.utils import PaddingStrategy

# --- Configuration & Argument Parsing ---
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")





def parse_args():
    parser = argparse.ArgumentParser(description="Train mBART model for Saudi Sign Language Translation")

    # Update paths for Colab environment
    parser.add_argument("--data_dir", type=str, default="/content/drive/MyDrive/saudi/saudi-signfor-all-competition",
                      help="Directory containing H5/CSV/JSON data files (Colab Path)")
    parser.add_argument("--output_dir", type=str, default="/content/drive/MyDrive/saudi/output",
                      help="Directory to save trained models and logs (Colab Path)")
    parser.add_argument("--results_dir", type=str, default="/content/drive/MyDrive/saudi/results",
                      help="Directory to save evaluation results and predictions (Colab Path)")

    # File names exactly as provided
    parser.add_argument("--train_h5_file", type=str, default="SSL.keypoints.train_signers_train_sentences.0.h5",
                       help="Filename of training H5 file (within data_dir)")
    parser.add_argument("--train_csv_file", type=str, default="SSL.keypoints.train_signers_train_sentences.csv",
                       help="Filename of training CSV file (within data_dir)")
    parser.add_argument("--test_h5_file", type=str, default="SSL.keypoints.test_signers_test_sentences.h5",
                       help="Filename of test H5 file (within data_dir)")
    parser.add_argument("--test_json_file", type=str, default="SSL.keypoints.test_signers_test_sentences.json",
                       help="Filename of test JSON file (within data_dir)")

    parser.add_argument("--seed", type=int, default=42, help="Random seed for reproducibility")
    parser.add_argument("--mbart_model_name", type=str, default="facebook/mbart-large-50-many-to-many-mmt", help="Pre-trained mBART model name")
    parser.add_argument("--target_lang", type=str, default="ar_AR", help="mBART language code for Arabic")
    parser.add_argument("--feature_dim", type=int, default=208, help="Keypoint feature dimension (check H5 files)")
    parser.add_argument("--epochs", type=int, default=15, help="Number of training epochs")
    parser.add_argument("--warmup_steps", type=int, default=500, help="Number of warmup steps for learning rate scheduler")
    parser.add_argument("--weight_decay", type=float, default=0.01, help="Weight decay for optimizer")
    parser.add_argument("--logging_steps", type=int, default=100, help="Log training info every N steps")
    parser.add_argument("--save_total_limit", type=int, default=2, help="Maximum number of checkpoints to save")
    parser.add_argument("--fp16", action="store_true", default=True, help="Enable FP16 training")

    # A100 GPU optimizations
    parser.add_argument("--gradient_checkpointing", action="store_true", default=False, help="Enable gradient checkpointing")
    parser.add_argument("--max_seq_length", type=int, default=512, help="Max sequence length for keypoints")
    parser.add_argument("--max_label_length", type=int, default=128, help="Max sequence length for target Arabic text")
    parser.add_argument("--train_batch_size", type=int, default=4, help="Training batch size per device")
    parser.add_argument("--eval_batch_size", type=int, default=16, help="Evaluation batch size per device")
    parser.add_argument("--gradient_accumulation_steps", type=int, default=1, help="Number of steps to accumulate gradients")
    parser.add_argument("--num_beams", type=int, default=5, help="Number of beams for generation during evaluation")

    # Use parse_known_args to ignore extra arguments from environments like Jupyter
    args, unknown = parser.parse_known_args()
    if unknown:
        print(f"Ignoring unrecognized arguments: {unknown}")

    # Calculate effective batch size
    num_devices = torch.cuda.device_count() if torch.cuda.is_available() else 1
    effective_batch_size = args.train_batch_size * args.gradient_accumulation_steps * num_devices
    print(f"Effective batch size: {effective_batch_size} (train_bs={args.train_batch_size} * grad_accum={args.gradient_accumulation_steps} * devices={num_devices})")

    # Create directories if they don't exist
    os.makedirs(args.data_dir, exist_ok=True)
    os.makedirs(args.output_dir, exist_ok=True)
    os.makedirs(args.results_dir, exist_ok=True)

    return args






# --- Utility Functions ---
def set_seed(seed_value):
    """Set random seed for reproducibility across libraries."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    transformers_set_seed(seed_value)

def check_and_prompt_for_file(file_path, file_type):
    """Check if a file exists and prompt for an alternative path if not found."""
    if os.path.exists(file_path):
        print(f"Found {file_type} file: {file_path}")
        return file_path

    print(f"ERROR: {file_type} file not found at {file_path}")

    user_input = None
    while True:
        user_input = input(f"Please enter an alternative path for the {file_type} file or type 'skip' to bypass: ")

        if user_input.lower() == 'skip':
            print(f"Skipping {file_type} file")
            return None

        if os.path.exists(user_input):
            print(f"Found {file_type} file at: {user_input}")
            return user_input
        else:
            print(f"File not found at: {user_input}")

def preload_h5_info(file_path):
    """
    Scans an H5 file and extracts key information:
    - List of valid keys
    - Mapping of keys to sequence lengths
    - Feature dimension
    """
    keys = []
    lengths = {}
    feature_dim = None
    if not file_path or not os.path.exists(file_path):
        print(f"Error: H5 path {file_path} invalid or not found.")
        return keys, lengths, feature_dim
    try:
        with h5py.File(file_path, "r") as f:
            print(f"Scanning HDF5 {file_path}...")
            valid_keys = list(f.keys())
            for key in tqdm(valid_keys, desc="Preloading HDF5 info"):
                try:
                    item = f[key]
                    seq_len = 0
                    current_dim = 0

                    # Handle potential group structure
                    if isinstance(item, h5py.Group):
                        data_keys = list(item.keys())
                        if data_keys:
                            data = item[data_keys[0]]
                            if isinstance(data, h5py.Dataset):
                                seq_len = data.shape[0]
                                if len(data.shape) > 1:
                                    current_dim = data.shape[1]
                        else:
                             continue # Skip empty group
                    elif isinstance(item, h5py.Dataset):
                        seq_len = item.shape[0]
                        if len(item.shape) > 1:
                            current_dim = item.shape[1]
                    else:
                        continue # Skip if not a group or dataset

                    if seq_len > 0:
                        keys.append(key)
                        lengths[key] = seq_len
                        if feature_dim is None and current_dim > 0:
                            feature_dim = current_dim
                        elif feature_dim is not None and current_dim != feature_dim and current_dim > 0:
                             pass

                except Exception as e:
                    print(f"Error processing key {key}: {e}")

            print(f"Found {len(keys)} non-empty keys in HDF5.")
            if feature_dim is not None:
                print(f"Detected primary feature dimension: {feature_dim}")
            else:
                print("Warning: Could not detect feature dimension from H5 file.")

            return keys, lengths, feature_dim
    except Exception as e:
        print(f"Error opening HDF5 {file_path}: {e}")
        return keys, lengths, feature_dim

def extract_signer_sentence_ids(key):
    """
    Extract signer ID and sentence ID from the key format (e.g., '00_0001').

    Args:
        key (str): The key in format 'XX_YYYY'

    Returns:
        tuple: (signer_id, sentence_id) or (None, None) if format doesn't match
    """
    try:
        if '_' in key:
            parts = key.split('_')
            if len(parts) == 2:
                signer_id = parts[0]
                sentence_id = parts[1]
                return signer_id, sentence_id
    except Exception:
        pass
    return None, None

def create_validation_splits_with_strict_separation(train_h5_keys, id_to_sentence_map, train_h5_lengths, max_seq_length, seed=42):
    """
    Create validation splits with STRICT separation from training data.
    This ensures validation data is NEVER used in training.

    Args:
        train_h5_keys: List of keys from training H5 file
        id_to_sentence_map: Mapping from keys to sentence translations
        train_h5_lengths: Mapping from keys to sequence lengths
        max_seq_length: Maximum sequence length to include
        seed: Random seed for reproducibility

    Returns:
        dict: Dictionary of validation splits and metadata
    """
    print("\n--- Creating Validation Splits with STRICT Separation ---")

    # Set random seed for reproducibility
    random.seed(seed)
    np.random.seed(seed)

    # Filter keys by length and valid translations
    valid_keys = [key for key in train_h5_keys
                  if key in id_to_sentence_map
                  and id_to_sentence_map[key]
                  and isinstance(id_to_sentence_map[key], str)
                  and id_to_sentence_map[key].strip()
                  and key in train_h5_lengths
                  and train_h5_lengths[key] <= max_seq_length]

    print(f"Valid keys after filtering: {len(valid_keys)}")

    # Extract signer and sentence information
    signer_to_keys = {}
    sentence_to_keys = {}
    key_info = {}

    for key in valid_keys:
        signer_id, sentence_id = extract_signer_sentence_ids(key)
        sentence = id_to_sentence_map[key]

        if signer_id is not None:
            if signer_id not in signer_to_keys:
                signer_to_keys[signer_id] = []
            signer_to_keys[signer_id].append(key)

        if sentence not in sentence_to_keys:
            sentence_to_keys[sentence] = []
        sentence_to_keys[sentence].append(key)

        key_info[key] = {
            'signer_id': signer_id,
            'sentence_id': sentence_id,
            'sentence': sentence
        }

    unique_signers = list(signer_to_keys.keys())
    unique_sentences = list(sentence_to_keys.keys())

    print(f"Found {len(unique_signers)} unique signers and {len(unique_sentences)} unique sentences")

    # Create validation splits with NO OVERLAP with training
    splits = {}

    # Split 1: Standard hold-out (completely random keys)
    train_standard, val_standard = train_test_split(
        valid_keys, test_size=0.1, random_state=seed)

    # Ensure strict separation
    assert set(train_standard).isdisjoint(set(val_standard)), "ERROR: Train and validation sets overlap!"

    splits['standard'] = {
        'name': 'Standard Random Split',
        'description': '10% of data randomly selected, completely held out from training',
        'train_keys': train_standard,
        'val_keys': val_standard,
        'challenge_level': 1
    }

    print(f"Standard split: {len(train_standard)} train, {len(val_standard)} validation samples")
    print(f"Verified: Train and validation sets have NO overlap")

    # Split 2: Unseen sentences
    if len(unique_sentences) >= 10:
        # Hold out 10% of sentences completely
        val_sentence_count = max(int(len(unique_sentences) * 0.1), 5)  # At least 5 sentences
        val_sentences = set(random.sample(unique_sentences, k=val_sentence_count))

        # Ensure sentences in validation are NEVER in training
        val_unseen_sentence_keys = []
        train_seen_sentence_keys = []

        for key in valid_keys:
            sentence = id_to_sentence_map[key]
            if sentence in val_sentences:
                val_unseen_sentence_keys.append(key)
            else:
                train_seen_sentence_keys.append(key)

        # Verify strict separation
        assert set(train_seen_sentence_keys).isdisjoint(set(val_unseen_sentence_keys)), "ERROR: Train and unseen sentence validation sets overlap!"

        splits['unseen_sentences'] = {
            'name': 'Unseen Sentences Split',
            'description': f'Hold out {len(val_sentences)} complete sentences (never seen during training)',
            'train_keys': train_seen_sentence_keys,
            'val_keys': val_unseen_sentence_keys,
            'challenge_level': 2,
            'val_sentences': val_sentences  # Store these for reference
        }

        print(f"Unseen sentences split: {len(train_seen_sentence_keys)} train, {len(val_unseen_sentence_keys)} validation samples")
        print(f"Verified: Train and validation sets have NO overlap")

    # Split 3: Unseen signers
    if len(unique_signers) >= 5:
        # Hold out 20% of signers completely
        val_signer_count = max(int(len(unique_signers) * 0.2), 2)  # At least 2 signers
        val_signers = set(random.sample(unique_signers, k=val_signer_count))

        # Ensure signers in validation are NEVER in training
        val_unseen_signer_keys = []
        train_seen_signer_keys = []

        for key in valid_keys:
            signer_id, _ = extract_signer_sentence_ids(key)
            if signer_id in val_signers:
                val_unseen_signer_keys.append(key)
            else:
                train_seen_signer_keys.append(key)

        # Verify strict separation
        assert set(train_seen_signer_keys).isdisjoint(set(val_unseen_signer_keys)), "ERROR: Train and unseen signer validation sets overlap!"

        splits['unseen_signers'] = {
            'name': 'Unseen Signers Split',
            'description': f'Hold out {len(val_signers)} complete signers (never seen during training)',
            'train_keys': train_seen_signer_keys,
            'val_keys': val_unseen_signer_keys,
            'challenge_level': 3,
            'val_signers': val_signers  # Store these for reference
        }

        print(f"Unseen signers split: {len(train_seen_signer_keys)} train, {len(val_unseen_signer_keys)} validation samples")
        print(f"Verified: Train and validation sets have NO overlap")

    # Split 4: Test-like challenge (both unseen signers and sentences)
    if 'unseen_signers' in splits and 'unseen_sentences' in splits:
        # Extract a different set of signers and sentences
        # We need to ensure we don't reuse the ones already held out
        used_val_signers = set(splits['unseen_signers']['val_signers'])
        used_val_sentences = set(splits['unseen_sentences']['val_sentences'])

        remaining_signers = [s for s in unique_signers if s not in used_val_signers]
        remaining_sentences = [s for s in unique_sentences if s not in used_val_sentences]

        if len(remaining_signers) >= 2 and len(remaining_sentences) >= 5:
            challenge_signer_count = min(val_signer_count, len(remaining_signers))
            challenge_sentence_count = min(val_sentence_count, len(remaining_sentences))

            challenge_signers = set(random.sample(remaining_signers, k=challenge_signer_count))
            challenge_sentences = set(random.sample(remaining_sentences, k=challenge_sentence_count))

            # Find keys that have EITHER challenge signers OR challenge sentences
            # This creates a strict train/val separation while ensuring enough validation samples
            val_challenge_keys = []
            train_challenge_keys = []

            for key in valid_keys:
                signer_id, _ = extract_signer_sentence_ids(key)
                sentence = id_to_sentence_map[key]

                # If this key has either a challenge signer or sentence, it goes to validation
                if (signer_id in challenge_signers) or (sentence in challenge_sentences):
                    val_challenge_keys.append(key)
                else:
                    train_challenge_keys.append(key)

            # Verify strict separation
            assert set(train_challenge_keys).isdisjoint(set(val_challenge_keys)), "ERROR: Train and challenge validation sets overlap!"

            # Only create this split if we have enough validation samples
            if len(val_challenge_keys) >= 10:
                splits['challenge'] = {
                    'name': 'Test-Like Challenge Split',
                    'description': 'Both unseen signers and unseen sentences (strictly held out from training)',
                    'train_keys': train_challenge_keys,
                    'val_keys': val_challenge_keys,
                    'challenge_level': 4,
                    'val_signers': challenge_signers,
                    'val_sentences': challenge_sentences
                }

                print(f"Challenge split: {len(train_challenge_keys)} train, {len(val_challenge_keys)} validation samples")
                print(f"Verified: Train and validation sets have NO overlap")

    return splits, key_info


def verify_split_separation(splits):
    """
    Verify that each split maintains strict separation between its own training and validation data.

    Args:
        splits: Dictionary of validation splits

    Returns:
        bool: True if all individual splits have strict separation, False otherwise
    """
    print("\n--- Verifying Individual Split Separation ---")
    all_valid = True

    # Check each split for strict separation between its own train and validation sets
    for name, split_info in splits.items():
        train_keys = set(split_info['train_keys'])
        val_keys = set(split_info['val_keys'])

        # Verify no overlap between train and val within this split
        if not train_keys.isdisjoint(val_keys):
            overlap = len(train_keys.intersection(val_keys))
            print(f"ERROR: Split '{name}' has {overlap} overlapping keys between train and validation!")
            all_valid = False
        else:
            print(f"✓ Split '{name}' has strict separation between train and validation sets")

    # Now provide info about cross-split overlap (not an error, just informational)
    print("\n--- Cross-Split Overlap Information (Expected) ---")
    split_names = list(splits.keys())
    for i, name1 in enumerate(split_names):
        for j in range(i+1, len(split_names)):
            name2 = split_names[j]
            train_keys1 = set(splits[name1]['train_keys'])
            val_keys1 = set(splits[name1]['val_keys'])
            train_keys2 = set(splits[name2]['train_keys'])
            val_keys2 = set(splits[name2]['val_keys'])

            train_train_overlap = len(train_keys1.intersection(train_keys2))
            val_val_overlap = len(val_keys1.intersection(val_keys2))

            # Show overlap info for training and validation sets
            print(f"Info: Splits '{name1}' and '{name2}':")
            print(f"  - {train_train_overlap} overlapping training keys")
            print(f"  - {val_val_overlap} overlapping validation keys")

            # Also show cross-overlap (informational only)
            train1_val2_overlap = len(train_keys1.intersection(val_keys2))
            train2_val1_overlap = len(train_keys2.intersection(val_keys1))

            print(f"  - {train1_val2_overlap} keys in both '{name1}' train and '{name2}' validation")
            print(f"  - {train2_val1_overlap} keys in both '{name2}' train and '{name1}' validation")

    if all_valid:
        print("\n✓ All individual splits maintain strict separation between their own train and validation sets!")
        print("NOTE: The overlap between different splits is normal and expected.")
        print("The training will use only ONE split (the most challenging one) for both training and validation.")
    else:
        print("\n✗ One or more splits have internal train/validation overlap!")

    return all_valid




def load_and_prepare_test_data(test_h5_path, test_json_path, test_h5_keys=None, test_h5_lengths=None):
    """
    Load and prepare the official test dataset

    Args:
        test_h5_path: Path to test H5 file
        test_json_path: Path to test JSON file
        test_h5_keys: List of keys from test H5 file (if already loaded)
        test_h5_lengths: Mapping from keys to sequence lengths (if already loaded)

    Returns:
        list: List of valid test keys
    """
    test_ids_from_json = []

    # Load test IDs from JSON
    if test_json_path and os.path.exists(test_json_path):
        try:
            with open(test_json_path, "r") as f:
                test_data_json = json.load(f)
                if isinstance(test_data_json, list) and test_data_json and isinstance(test_data_json[0], dict) and "id" in test_data_json[0]:
                    test_ids_from_json = [item["id"] for item in test_data_json]
                    print(f"Loaded {len(test_ids_from_json)} test IDs from {test_json_path}")
                else:
                    print(f"Warning: Test JSON format not recognized or empty at {test_json_path}.")
        except Exception as e:
            print(f"Error reading test JSON {test_json_path}: {e}")

    # If we don't have keys/lengths yet, load them from H5
    if test_h5_keys is None or test_h5_lengths is None:
        if test_h5_path and os.path.exists(test_h5_path):
            test_h5_keys, test_h5_lengths, _ = preload_h5_info(test_h5_path)
        else:
            print(f"Warning: Test H5 file not found at {test_h5_path}")
            return []

    # Find keys that exist in both H5 and JSON (if JSON was loaded)
    valid_test_keys = []
    if test_ids_from_json:
        valid_test_keys = [key for key in test_h5_keys if key in test_ids_from_json and key in test_h5_lengths]
    else:
        valid_test_keys = [key for key in test_h5_keys if key in test_h5_lengths]

    print(f"Found {len(valid_test_keys)} valid test keys for evaluation")
    return valid_test_keys

# --- Dataset Class ---
class SignDataset(Dataset):
    def __init__(self, h5_file_path, data_keys_to_use, id_to_sentence_map,
                 tokenizer, h5_lengths_map, max_seq_length,
                 max_label_length, feature_dim, target_lang, is_test_set=False):
        self.h5_file_path = h5_file_path
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.max_label_length = max_label_length
        self.feature_dim = feature_dim
        self.target_lang = target_lang
        self.is_test_set = is_test_set
        self.id_to_sentence_map = id_to_sentence_map
        self.h5_lengths_map = h5_lengths_map

        self.data_info = []
        print(f"Initializing dataset ({'Test' if is_test_set else 'Train/Val'}). Processing {len(data_keys_to_use)} provided keys...")

        skipped_count = 0
        skipped_length = 0
        skipped_missing_label = 0
        skipped_zero_length = 0
        for key in tqdm(data_keys_to_use, desc="Preparing dataset samples"):
            original_length = self.h5_lengths_map.get(key)
            sentence = self.id_to_sentence_map.get(key) if not self.is_test_set else key # Use key as placeholder for test

            # Validation (ensure sentence exists unless it's test set)
            if sentence is None and not self.is_test_set:
                 skipped_missing_label += 1
                 skipped_count += 1
                 continue
            # Skip empty sentences in train/val
            if isinstance(sentence, str) and not sentence.strip() and not self.is_test_set:
                 skipped_missing_label += 1
                 skipped_count += 1
                 continue

            # Length validation
            if original_length is None or original_length == 0 :
                skipped_zero_length += 1
                skipped_count += 1
                continue
            if original_length > self.max_seq_length:
                skipped_length += 1
                skipped_count += 1
                continue # Skip sequences longer than max_seq_length

            self.data_info.append((key, sentence, original_length))

        print(f"Finished preparing dataset: {len(self.data_info)} samples included.")
        print(f"  Skipped {skipped_count} samples total ({skipped_length} due to length > {self.max_seq_length}, {skipped_zero_length} zero length, {skipped_missing_label} missing/empty label).")
        if not self.data_info:
            print("WARNING: Dataset is empty after processing.")

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        key, sentence, original_length = self.data_info[idx]
        features = None
        actual_feature_dim = self.feature_dim # Assume this unless proven otherwise

        try:
            with h5py.File(self.h5_file_path, "r") as f:
                item = f[key]
                # Check if item is a Group or Dataset
                if isinstance(item, h5py.Group):
                    data_keys = list(item.keys())
                    if data_keys:
                        data_to_load = item[data_keys[0]]
                    else:
                        raise ValueError(f"Group {key} is empty")
                elif isinstance(item, h5py.Dataset):
                    data_to_load = item
                else:
                    raise ValueError(f"Key {key} is not a Group or Dataset")

                if data_to_load is not None and isinstance(data_to_load, h5py.Dataset):
                    features_np = data_to_load[()] # Load data into numpy array

                    # Handle potential dimension mismatch if possible (e.g., flattened)
                    current_dim = features_np.shape[1] if len(features_np.shape) > 1 else 0
                    if len(features_np.shape) < 2 or current_dim == 0:
                        if len(features_np.shape) == 1 and self.feature_dim > 0 and features_np.shape[0] % self.feature_dim == 0:
                            seq_len = features_np.shape[0] // self.feature_dim
                            features_np = features_np.reshape(seq_len, self.feature_dim)
                            original_length = seq_len # Update original length based on reshape
                            current_dim = self.feature_dim
                        else:
                            raise ValueError(f"Unexpected shape or zero dim for key {key}. Expected (*, {self.feature_dim}), Got {features_np.shape}")
                    elif current_dim != self.feature_dim:
                         raise ValueError(f"Feature dim mismatch for key {key}. Expected {self.feature_dim}, Got {current_dim}")

                    features = torch.tensor(features_np, dtype=torch.float32)
                else:
                    raise ValueError(f"No valid dataset found for key {key}")

        except Exception as e:
             print(f"Error loading key {key} from {self.h5_file_path}: {e}")
             raise RuntimeError(f"Failed to load data for key {key}") from e

        assert original_length <= self.max_seq_length, f"Key {key} has length {original_length} > max_seq_length {self.max_seq_length} despite filtering."

        # Padding
        pad_value = 0.0
        num_padding = self.max_seq_length - original_length

        if num_padding > 0:
            padded_features = F.pad(features, (0, 0, 0, num_padding), value=pad_value)
        else: # num_padding should be 0 here
            padded_features = features

        # Create attention mask for features (1 for real data, 0 for padding)
        feature_attention_mask = torch.zeros(self.max_seq_length, dtype=torch.long)
        feature_attention_mask[:original_length] = 1

        # Prepare labels
        if self.is_test_set:
            # FIX: Create dummy inputs/labels with proper length instead of just length 1
            # Use max_label_length instead of a hardcoded short length
            input_ids = torch.full((self.max_label_length,), self.tokenizer.pad_token_id, dtype=torch.long)
            label_attention_mask = torch.zeros(self.max_label_length, dtype=torch.long)
            # Set first position to non-padding
            input_ids[0] = self.tokenizer.bos_token_id if hasattr(self.tokenizer, 'bos_token_id') else self.tokenizer.pad_token_id
            label_attention_mask[0] = 1
            # Create labels with proper length (-100 for loss masking)
            labels_output = input_ids.clone()
            labels_output[labels_output == self.tokenizer.pad_token_id] = -100
        else:
            self.tokenizer.tgt_lang = self.target_lang
            tokenized_output = self.tokenizer(
                str(sentence), # Ensure it's a string
                truncation=True,
                padding="max_length",
                max_length=self.max_label_length,
                return_tensors="pt"
            )
            input_ids = tokenized_output["input_ids"].squeeze(0) # Remove batch dim
            label_attention_mask = tokenized_output["attention_mask"].squeeze(0)

            # Create labels for loss calculation (-100 for padding)
            labels_output = input_ids.clone()
            labels_output[labels_output == self.tokenizer.pad_token_id] = -100

        return {
            "input_features": padded_features,
            "attention_mask": feature_attention_mask,
            "labels": labels_output, # Needs to be present, even if dummy for test
            "decoder_attention_mask": label_attention_mask,
            "key": key # Keep key for potential debugging/mapping later
        }

def create_datasets_from_splits(splits, train_h5_path, id_to_sentence_map, train_h5_lengths,
                               tokenizer, max_seq_length, max_label_length, feature_dim, target_lang,
                               test_h5_path=None, test_h5_lengths=None):
    """
    Create training and validation datasets from the splits.

    Args:
        splits: Dictionary of validation splits
        train_h5_path: Path to training H5 file
        id_to_sentence_map: Mapping from keys to sentence translations
        train_h5_lengths: Mapping from keys to sequence lengths
        tokenizer: The tokenizer
        max_seq_length: Maximum sequence length
        max_label_length: Maximum label length
        feature_dim: Feature dimension
        target_lang: Target language
        test_h5_path: Path to test H5 file
        test_h5_lengths: Mapping from test keys to sequence lengths

    Returns:
        dict: Dictionary of datasets for each split
    """
    print("\n--- Creating Datasets from Splits ---")

    datasets = {}

    for split_name, split_info in splits.items():
        print(f"\nCreating datasets for '{split_info['name']}' split:")

        train_keys = split_info['train_keys']
        val_keys = split_info['val_keys']

        # Determine if this is using test data
        is_test_split = 'test' in split_name and test_h5_path is not None
        h5_path = test_h5_path if is_test_split else train_h5_path
        h5_lengths = test_h5_lengths if is_test_split else train_h5_lengths

        # For test validation, we don't have translations
        empty_map = {}

        # Create training dataset
        train_dataset = None
        if not is_test_split and train_keys:
            try:
                train_dataset = SignDataset(
                    h5_file_path=h5_path,
                    data_keys_to_use=train_keys,
                    id_to_sentence_map=id_to_sentence_map,
                    tokenizer=tokenizer,
                    h5_lengths_map=h5_lengths,
                    max_seq_length=max_seq_length,
                    max_label_length=max_label_length,
                    feature_dim=feature_dim,
                    target_lang=target_lang,
                    is_test_set=False
                )
                print(f"  Created training dataset with {len(train_dataset)} samples")
            except Exception as e:
                print(f"  Error creating training dataset: {e}")

        # Create validation dataset
        val_dataset = None
        if val_keys:
            try:
                val_dataset = SignDataset(
                    h5_file_path=h5_path,
                    data_keys_to_use=val_keys,
                    # Use empty map for test validation to mark as test data
                    id_to_sentence_map=empty_map if is_test_split else id_to_sentence_map,
                    tokenizer=tokenizer,
                    h5_lengths_map=h5_lengths,
                    max_seq_length=max_seq_length,
                    max_label_length=max_label_length,
                    feature_dim=feature_dim,
                    target_lang=target_lang,
                    is_test_set=is_test_split
                )
                print(f"  Created validation dataset with {len(val_dataset)} samples")
            except Exception as e:
                print(f"  Error creating validation dataset: {e}")

        datasets[split_name] = {
            'train': train_dataset,
            'validation': val_dataset,
            'info': split_info
        }

    return datasets

# --- Custom Data Collator --- #
@dataclass
class CustomDataCollator:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None # Not used for features here, padding done in dataset
    pad_to_multiple_of: Optional[int] = None # Applied to labels
    label_pad_token_id: int = -100
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        input_features = [f["input_features"] for f in features]
        attention_mask = [f["attention_mask"] for f in features]
        labels = [f["labels"] for f in features] # These are the ones to pad dynamically
        decoder_attention_mask = [f["decoder_attention_mask"] for f in features]

        batch_input_features = torch.stack(input_features)
        batch_attention_mask = torch.stack(attention_mask)

        max_label_len = max(len(lab) for lab in labels)
        if self.pad_to_multiple_of is not None:
            max_label_len = (
                (max_label_len + self.pad_to_multiple_of - 1)
                // self.pad_to_multiple_of
                * self.pad_to_multiple_of
            )

        padded_labels = []
        padded_decoder_attention_mask = []
        for lab, dec_attn in zip(labels, decoder_attention_mask):
            remainder = max_label_len - len(lab)
            padded_labels.append(F.pad(lab, (0, remainder), value=self.label_pad_token_id))
            padded_decoder_attention_mask.append(F.pad(dec_attn, (0, remainder), value=0))

        batch_labels = torch.stack(padded_labels)
        batch_decoder_attention_mask = torch.stack(padded_decoder_attention_mask)

        batch = {
            "input_features": batch_input_features,
            "attention_mask": batch_attention_mask,
            "labels": batch_labels,
            "decoder_attention_mask": batch_decoder_attention_mask,
        }
        return batch

# --- Model Definition --- #
class VisualHead(nn.Module):
    def __init__(self, input_dim, output_dim, dropout_rate=0.1):
        super().__init__()
        self.projection = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.GELU(),
            nn.Dropout(dropout_rate)
        )
        print(f"Initialized VisualHead: Linear({input_dim} -> {output_dim})")

    def forward(self, features):
        projected_features = self.projection(features)
        return projected_features

class SignTranslationModel(nn.Module):
    def __init__(self, feature_dim, mbert_model_name, dropout_rate=0.1):
        super().__init__()
        print(f"Initializing SignTranslationModel:")
        print(f"  Loading Base mBART Model From: {mbert_model_name}")
        print(f"  Feature Dim (Input): {feature_dim}")

        try:
            # CRUCIAL CHANGE: Set config before loading model
            from transformers import MBartConfig
            config = MBartConfig.from_pretrained(mbert_model_name)

            # Disable SDPA in config before loading
            config._attn_implementation = "eager"

            # Now load with our config
            self.mbart = MBartForConditionalGeneration.from_pretrained(
                mbert_model_name,
                config=config
            )
            print(f"  Base mBART model loaded successfully with eager attention.")
        except Exception as e:
            print(f"ERROR loading base mBART model from {mbert_model_name}: {e}")
            raise

        self.model_hidden_dim = self.mbart.config.d_model
        print(f"  Model Hidden Dim (d_model): {self.model_hidden_dim}")

        self.visual_head = VisualHead(feature_dim, self.model_hidden_dim, dropout_rate)

        # Initialize generation_config
        self.generation_config = None  # Will be set later in training setup

    def forward(self, input_features, attention_mask, labels=None, decoder_attention_mask=None, **kwargs):
        input_features = input_features.float()
        try:
            inputs_embeds = self.visual_head(input_features)
        except Exception as e:
            print(f"VisualHead projection error: {e}\nInput shape: {input_features.shape}, Input dtype: {input_features.dtype}")
            raise

        try:
            outputs = self.mbart(
                inputs_embeds=inputs_embeds,
                attention_mask=attention_mask,
                labels=labels,
                decoder_attention_mask=decoder_attention_mask,
                return_dict=True
            )
        except Exception as e:
            print(f"mBART forward error: {e}")
            print(f"  inputs_embeds shape: {inputs_embeds.shape if inputs_embeds is not None else 'None'}, dtype: {inputs_embeds.dtype if inputs_embeds is not None else 'None'}")
            print(f"  attention_mask shape: {attention_mask.shape if attention_mask is not None else 'None'}, dtype: {attention_mask.dtype if attention_mask is not None else 'None'}")
            print(f"  labels shape: {labels.shape if labels is not None else 'None'}, dtype: {labels.dtype if labels is not None else 'None'}")
            print(f"  decoder_attention_mask shape: {decoder_attention_mask.shape if decoder_attention_mask is not None else 'None'}, dtype: {decoder_attention_mask.dtype if decoder_attention_mask is not None else 'None'}")
            raise
        return outputs

    @torch.no_grad()
    def generate(self, input_features, attention_mask, **generate_kwargs):
        self.eval()
        input_features = input_features.float() # Ensure float input
        inputs_embeds = self.visual_head(input_features)

        # ATTENTION MASK FIX - Ensure correct shape for generate
        # During beam search, the attention mask shape can cause issues
        # We need to ensure it's correctly shaped for the mBART model

        if attention_mask is not None:
            # Get batch size
            batch_size = attention_mask.size(0)

            # Ensure attention mask is the correct shape by reshaping
            attention_mask = attention_mask.view(batch_size, -1)

            # Debug log the shape
            # print(f"Generate: Attention mask reshaped to {attention_mask.shape}")

        decoder_start_token_id = self.mbart.config.forced_bos_token_id # Default
        if "tgt_lang" in generate_kwargs:
            try:
                lang_code = generate_kwargs["tgt_lang"]
                lang_token_id = global_tokenizer.lang_code_to_id.get(lang_code)
                if lang_token_id is not None:
                   decoder_start_token_id = lang_token_id
                else:
                    print(f"Warning: Target language '{lang_code}' not found in tokenizer lang_code_to_id. Using default BOS: {decoder_start_token_id}.")
            except Exception as e:
                 print(f"Warning: Error accessing tokenizer for target language '{generate_kwargs['tgt_lang']}': {e}. Using default BOS: {decoder_start_token_id}")

        # Use the model's generation_config if available, otherwise use defaults
        if hasattr(self, 'generation_config') and self.generation_config is not None:
            # Clone and update the generation config
            from copy import deepcopy
            gen_config = deepcopy(self.generation_config)

            # Update with any kwargs provided
            for k, v in generate_kwargs.items():
                if hasattr(gen_config, k):
                    setattr(gen_config, k, v)

            # Extra safety check for critical parameters
            if "max_length" in generate_kwargs:
                gen_config.max_length = generate_kwargs["max_length"]
            if "num_beams" in generate_kwargs:
                gen_config.num_beams = generate_kwargs["num_beams"]
            if "decoder_start_token_id" in generate_kwargs:
                gen_config.decoder_start_token_id = generate_kwargs["decoder_start_token_id"]
            else:
                gen_config.decoder_start_token_id = decoder_start_token_id

            # Run generation with the config object
            try:
                return self.mbart.generate(
                    inputs_embeds=inputs_embeds,
                    attention_mask=attention_mask,
                    generation_config=gen_config
                )
            except Exception as e:
                print(f"Generate failed with generation_config: {e}")
                print("Falling back to kwargs method...")

        # Fall back to kwargs method
        final_generate_kwargs = {
            "max_length": generate_kwargs.get("max_length", self.mbart.config.max_length),
            "num_beams": generate_kwargs.get("num_beams", self.mbart.config.num_beams),
            "early_stopping": generate_kwargs.get("early_stopping", True),
            "decoder_start_token_id": decoder_start_token_id,
        }
        for k, v in generate_kwargs.items():
            if k not in ["tgt_lang", "max_length", "num_beams", "early_stopping", "decoder_start_token_id", "forced_decoder_ids"]:
                 final_generate_kwargs[k] = v

        return self.mbart.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            **final_generate_kwargs
        )

    def save_pretrained(self, save_dir):
        """Save model to the specified directory."""
        os.makedirs(save_dir, exist_ok=True)

        # Save mBART config
        self.mbart.config.save_pretrained(save_dir)

        # Save model state_dict
        torch.save(self.state_dict(), os.path.join(save_dir, "pytorch_model.bin"))

        # Save generation config if it exists
        if hasattr(self, 'generation_config') and self.generation_config is not None:
            self.generation_config.save_pretrained(save_dir)

        print(f"Model saved to {save_dir}")

    @classmethod
    def from_pretrained(cls, load_dir, device=None):
        """Load model from the specified directory."""
        # Load config
        from transformers import MBartConfig
        config = MBartConfig.from_pretrained(load_dir)

        # Determine feature_dim (needed to initialize the model)
        # This might be stored in the config or we need to infer it
        feature_dim = getattr(config, "feature_dim", 208)  # Default to 208 if not found

        # Create new model instance
        model = cls(
            feature_dim=feature_dim,
            mbert_model_name=load_dir
        )

        # Load state_dict
        state_dict_path = os.path.join(load_dir, "pytorch_model.bin")
        if os.path.exists(state_dict_path):
            state_dict = torch.load(state_dict_path, map_location="cpu")
            model.load_state_dict(state_dict)
            print(f"Model weights loaded from {state_dict_path}")
        else:
            print(f"Warning: No model weights found at {state_dict_path}")

        # Load generation config if it exists
        generation_config_path = os.path.join(load_dir, "generation_config.json")
        if os.path.exists(generation_config_path):
            model.generation_config = GenerationConfig.from_pretrained(load_dir)
            print(f"Generation config loaded from {generation_config_path}")

        # Move to device if specified
        if device is not None:
            model = model.to(device)

        return model

    def gradient_checkpointing_enable(self, gradient_checkpointing_kwargs=None):
        """Enable gradient checkpointing for memory efficiency."""
        print("Enabling gradient checkpointing on underlying mBART model...")
        if hasattr(self.mbart, "gradient_checkpointing_enable"):
            self.mbart.gradient_checkpointing_enable(gradient_checkpointing_kwargs=gradient_checkpointing_kwargs)
            print("Gradient checkpointing enabled on mBART.")
        else:
            print("Warning: Underlying mBART model does not have 'gradient_checkpointing_enable' method.")

# --- Evaluation Metrics --- #
bleu_metric = evaluate.load("sacrebleu")
global_tokenizer = None # To be set after tokenizer initialization

def postprocess_text(preds, labels):
    """Process predictions and labels for evaluation."""
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    """Compute BLEU score for evaluation."""
    global global_tokenizer
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0] # Predictions are often tuples

    labels = np.where(labels != -100, labels, global_tokenizer.pad_token_id)

    try:
        decoded_preds = global_tokenizer.batch_decode(preds, skip_special_tokens=True)
    except Exception as e:
        print(f"Error decoding predictions: {e}")
        decoded_preds = ["<DECODE_ERROR>"] * len(preds)

    try:
        decoded_labels = global_tokenizer.batch_decode(labels, skip_special_tokens=True)
    except Exception as e:
        print(f"Error decoding labels: {e}")
        decoded_labels = ["<DECODE_ERROR>"] * len(labels)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    try:
        if not isinstance(decoded_preds, list) or not all(isinstance(p, str) for p in decoded_preds):
             raise ValueError("Decoded predictions are not in the expected format (List[str]).")
        if not isinstance(decoded_labels, list) or not all(isinstance(ref, list) and all(isinstance(s, str) for s in ref) for ref in decoded_labels):
            raise ValueError("Decoded labels are not in the expected format (List[List[str]]).")

        result = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"bleu": result["score"]}
    except Exception as e:
        print(f"Error computing BLEU: {e}")
        result = {"bleu": 0.0} # Default to 0 on error

    prediction_lens = [np.count_nonzero(pred != global_tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)

    result = {k: round(v, 4) for k, v in result.items()}
    return result

def train_and_evaluate_with_strict_separation(model, datasets, args, tokenizer, data_collator):
    """
    Train and evaluate the model with strict separation between training and validation.

    Args:
        model: The model to train
        datasets: Dictionary of datasets for each split
        args: Training arguments
        tokenizer: The tokenizer
        data_collator: The data collator

    Returns:
        The trained model
    """
    print("\n=== Training and Evaluation with Strict Separation ===")

    # Create results directory
    os.makedirs(args.results_dir, exist_ok=True)
    results_file = os.path.join(args.results_dir, "training_results.txt")

    # Select primary split for training (highest challenge level with valid datasets)
    available_splits = [(name, info['info']['challenge_level'])
                       for name, info in datasets.items()
                       if info['train'] is not None and info['validation'] is not None]

    if not available_splits:
        print("ERROR: No valid splits found for training. Aborting.")
        return model

    # Sort by challenge level (descending)
    available_splits.sort(key=lambda x: x[1], reverse=True)
    primary_split = available_splits[0][0]

    primary_train = datasets[primary_split]['train']
    primary_validation = datasets[primary_split]['validation']
    split_info = datasets[primary_split]['info']

    print(f"\nUsing '{split_info['name']}' as primary split for training.")
    print(f"Challenge level: {split_info['challenge_level']}")
    print(f"Description: {split_info['description']}")
    print(f"Training samples: {len(primary_train)}")
    print(f"Validation samples: {len(primary_validation)}")

    # Setup logging
    with open(results_file, "w") as f:
        f.write("=== Sign Language Translation Training Results ===\n\n")
        f.write(f"Model: mBART with Visual Head\n")
        f.write(f"Primary split: {split_info['name']}\n")
        f.write(f"Challenge level: {split_info['challenge_level']}\n")
        f.write(f"Description: {split_info['description']}\n")
        f.write(f"Training samples: {len(primary_train)}\n")
        f.write(f"Validation samples: {len(primary_validation)}\n\n")

        f.write("Other available splits:\n")
        for name, info in datasets.items():
            if name != primary_split and info['train'] is not None and info['validation'] is not None:
                f.write(f"- {info['info']['name']}: {len(info['train'])} train, {len(info['validation'])} validation\n")
                f.write(f"  {info['info']['description']}\n")

        f.write("\n=== TRAINING PROGRESS ===\n")

    # Setup target language tokens
    lang_token_id = tokenizer.lang_code_to_id.get(args.target_lang)
    if lang_token_id is None:
        print(f"Warning: Target language code '{args.target_lang}' not found in tokenizer's lang_code_to_id map.")
        lang_token_id = model.mbart.config.decoder_start_token_id
        print(f"Using fallback decoder_start_token_id: {lang_token_id}")

    # Generation config
    generation_config = GenerationConfig(
        max_length=args.max_label_length,
        num_beams=args.num_beams,
        early_stopping=True,
        decoder_start_token_id=lang_token_id,
        forced_bos_token_id=lang_token_id,
    )

    model.generation_config = generation_config

    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir=args.output_dir,
        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.train_batch_size,
        per_device_eval_batch_size=args.eval_batch_size,
        gradient_accumulation_steps=args.gradient_accumulation_steps,
        warmup_steps=args.warmup_steps,
        weight_decay=args.weight_decay,
        logging_dir=os.path.join(args.output_dir, "logs"),
        logging_steps=args.logging_steps,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=args.save_total_limit,
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        greater_is_better=True,
        predict_with_generate=True,
        generation_max_length=args.max_label_length,
        generation_num_beams=args.num_beams,
        fp16=args.fp16 and torch.cuda.is_available(),
        gradient_checkpointing=args.gradient_checkpointing,
        report_to=["tensorboard"],
        dataloader_num_workers=2,
        dataloader_pin_memory=True if torch.cuda.is_available() else False,
        save_safetensors=False,
    )

    # Initialize trainer with primary validation dataset
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=primary_train,
        eval_dataset=primary_validation,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # Early stopping setup
    early_stopping_patience = 3
    best_epoch = 0
    best_bleu = 0
    no_improvement_count = 0

    # Train the model
    print("\n=== Starting Training ===")

    # Check for existing checkpoints
    checkpoint_dir = args.output_dir
    resume_from_checkpoint = False

    # Look for checkpoint directories
    if os.path.exists(checkpoint_dir):
        checkpoint_dirs = [d for d in os.listdir(checkpoint_dir)
                          if os.path.isdir(os.path.join(checkpoint_dir, d))
                          and d.startswith("checkpoint-")]

        if checkpoint_dirs:
            # Sort checkpoints by step number
            checkpoint_dirs.sort(key=lambda x: int(x.split("-")[1]))
            latest_checkpoint = os.path.join(checkpoint_dir, checkpoint_dirs[-1])
            print(f"\nFound existing checkpoint: {latest_checkpoint}")
            resume_choice = input("Do you want to resume training from this checkpoint? (y/n): ").lower()

            if resume_choice == 'y':
                resume_from_checkpoint = latest_checkpoint
                print(f"Will resume training from: {resume_from_checkpoint}")
            else:
                print("Starting training from scratch.")
        else:
            print("No checkpoints found. Starting training from scratch.")

    for epoch in range(args.epochs):
        print(f"\n--- Epoch {epoch + 1}/{args.epochs} ---")

        # Train for one epoch - modified to support checkpoint resumption
        train_result = trainer.train(resume_from_checkpoint=resume_from_checkpoint)

        # Only use checkpoint for first epoch, then set to False
        if resume_from_checkpoint:
            resume_from_checkpoint = False

        train_metrics = train_result.metrics

        print(f"Training loss: {train_metrics['train_loss']:.4f}")

        # Evaluate on all validation splits
        with open(results_file, "a") as f:
            f.write(f"\nEpoch {epoch + 1}\n")
            f.write(f"Training loss: {train_metrics['train_loss']:.4f}\n")

        # Track best model
        current_bleu = None

        # Evaluate on each split's validation set
        for split_name, split_data in datasets.items():
            val_dataset = split_data['validation']

            if val_dataset is None:
                continue

            split_desc = split_data['info']['name']
            is_test_split = 'test' in split_name

            if is_test_split:
                # For test validation, we can only generate predictions, not evaluate
                print(f"\nGenerating predictions for '{split_desc}' validation")
                predictions = trainer.predict(
                    test_dataset=val_dataset,
                    metric_key_prefix=f"val_{split_name}",
                )

                with open(results_file, "a") as f:
                    f.write(f"{split_desc}: Generated predictions only (no reference translations)\n")
            else:
                # For regular validation, evaluate with metrics
                print(f"\nEvaluating on '{split_desc}' validation")
                val_metrics = trainer.evaluate(
                    eval_dataset=val_dataset,
                    metric_key_prefix=f"val_{split_name}",
                )

                bleu_score = val_metrics[f"val_{split_name}_bleu"]
                print(f"{split_desc} BLEU: {bleu_score:.2f}")

                with open(results_file, "a") as f:
                    f.write(f"{split_desc} BLEU: {bleu_score:.2f}\n")

                # Track primary split for early stopping
                if split_name == primary_split:
                    current_bleu = bleu_score

        # Check early stopping on primary validation
        if current_bleu is not None:
            if current_bleu > best_bleu:
                best_bleu = current_bleu
                best_epoch = epoch + 1
                no_improvement_count = 0

                # Save best model
                best_model_dir = os.path.join(args.output_dir, "best_model")
                os.makedirs(best_model_dir, exist_ok=True)
                model.save_pretrained(best_model_dir)
                tokenizer.save_pretrained(best_model_dir)

                print(f"New best model saved with BLEU: {best_bleu:.2f}")
            else:
                no_improvement_count += 1
                print(f"No improvement for {no_improvement_count} epochs (best: {best_bleu:.2f} at epoch {best_epoch})")

        # Check early stopping
        if no_improvement_count >= early_stopping_patience:
            print(f"\nEarly stopping after {epoch + 1} epochs!")
            with open(results_file, "a") as f:
                f.write(f"\nEarly stopping triggered. Best model was at epoch {best_epoch} with BLEU: {best_bleu:.2f}\n")
            break

    print("\n=== Training Complete ===")
    print(f"Best model was at epoch {best_epoch} with BLEU: {best_bleu:.2f}")

    # Load best model for final evaluation
    best_model_path = os.path.join(args.output_dir, "best_model")
    if os.path.exists(best_model_path):
        print(f"\nLoading best model from epoch {best_epoch}...")
        model = model.from_pretrained(best_model_path, device=device)

    # Final evaluation on all validation splits
    print("\n=== Final Evaluation ===")

    with open(results_file, "a") as f:
        f.write("\n=== FINAL EVALUATION ===\n")

    for split_name, split_data in datasets.items():
        val_dataset = split_data['validation']

        if val_dataset is None:
            continue

        split_desc = split_data['info']['name']
        is_test_split = 'test' in split_name

        if is_test_split:
            print(f"\nFinal prediction generation for '{split_desc}'")
            predict_results = trainer.predict(
                test_dataset=val_dataset,
                metric_key_prefix=f"final_{split_name}",
            )

            with open(results_file, "a") as f:
                f.write(f"Final {split_desc}: Generated predictions only\n")
        else:
            print(f"\nFinal evaluation on '{split_desc}'")
            val_metrics = trainer.evaluate(
                eval_dataset=val_dataset,
                metric_key_prefix=f"final_{split_name}",
            )

            bleu_score = val_metrics[f"final_{split_name}_bleu"]
            print(f"Final {split_desc} BLEU: {bleu_score:.2f}")

            with open(results_file, "a") as f:
                f.write(f"Final {split_desc} BLEU: {bleu_score:.2f}\n")

    # Generate predictions for official test set if available
    if 'official_test' in datasets and datasets['official_test']['validation'] is not None:
        test_dataset = datasets['official_test']['validation']

        print("\n=== Generating Official Test Set Predictions ===")

        predict_results = trainer.predict(
            test_dataset=test_dataset,
            metric_key_prefix="test",
        )

        # Save predictions
        predictions = predict_results.predictions
        predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
        decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        cleaned_predictions = [pred.strip() for pred in decoded_predictions]

        # Create submission format
        output_prediction_file = os.path.join(args.results_dir, "test_predictions.txt")
        with open(output_prediction_file, "w", encoding="utf-8") as writer:
            writer.write("ID\tPrediction\n")
            for idx, prediction in enumerate(cleaned_predictions):
                if idx < len(test_dataset.data_info):
                    sample_key = test_dataset.data_info[idx][0]
                    writer.write(f"{sample_key}\t{prediction}\n")

        print(f"Test predictions saved to {output_prediction_file}")

        # Create JSON format for submission
        submission_file = os.path.join(args.results_dir, "submission.json")
        submission_data = []

        for idx, prediction in enumerate(cleaned_predictions):
            if idx < len(test_dataset.data_info):
                sample_key = test_dataset.data_info[idx][0]
                submission_data.append({
                    "id": sample_key,
                    "translation": prediction
                })

        with open(submission_file, "w", encoding="utf-8") as f:
            json.dump(submission_data, f, ensure_ascii=False, indent=2)

        print(f"Submission file saved to {submission_file}")

    return model

In [ ]:
# === NEW: Main execution function ===
def main():
    # Create the system
    slt_system = SignLanguageTranslationSystem()

    # Step 1: Mount Drive
    slt_system.mount_drive()

    # Step 2: Setup paths and arguments
    slt_system.setup()

    # Step 3: Load data
    slt_system.load_data()

    # Step 4: Initialize tokenizer
    slt_system.initialize_tokenizer()

    # Step 5: Create datasets
    slt_system.create_datasets()

    # Step 6: Initialize model
    slt_system.initialize_model()

    # Step 7: Train model (this performs a single training run)
    slt_system.train_model()

    # Additional steps (uncomment as needed)
    slt_system.evaluate_model()  # Additional evaluation
    slt_system.generate_predictions()  # Generate test predictions

    print("All processing completed successfully!")

# === Entry point for direct execution ===
if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Ignoring unrecognized arguments: ['-f', '/root/.local/share/jupyter/runtime/kernel-38e1520b-38a4-4763-b504-4a0f5a3158e5.json']
Effective batch size: 4 (train_bs=4 * grad_accum=1 * devices=1)
Using device: cuda
Seed set to: 42
Found training H5 file: /content/drive/MyDrive/saudi/saudi-signfor-all-competition/SSL.keypoints.train_signers_train_sentences.0.h5
Found training CSV file: /content/drive/MyDrive/saudi/saudi-signfor-all-competition/SSL.keypoints.train_signers_train_sentences.csv
Found test H5 file: /content/drive/MyDrive/saudi/saudi-signfor-all-competition/SSL.keypoints.test_signers_test_sentences.h5
Found test JSON file: /content/drive/MyDrive/saudi/saudi-signfor-all-competition/SSL.keypoints.test_signers_test_sentences.json
Loaded training CSV: /content/drive/MyDrive/saudi/saudi-signfor-all-competition/SSL.keypoints.

Preloading HDF5 info: 100%|██████████| 24109/24109 [00:06<00:00, 3715.66it/s]


Found 24109 non-empty keys in HDF5.
Detected primary feature dimension: 208
Scanning HDF5 /content/drive/MyDrive/saudi/saudi-signfor-all-competition/SSL.keypoints.test_signers_test_sentences.h5...


Preloading HDF5 info: 100%|██████████| 200/200 [00:00<00:00, 3919.97it/s]


Found 200 non-empty keys in HDF5.
Detected primary feature dimension: 208

--- Training Sequence Length Analysis ---
Total sequences found in H5: 24109
Min length: 29
Max length: 1109
Mean length: 192.42
Median length: 161.0
90th percentile: 340.00
95th percentile: 408.00
99th percentile: 537.00
Num sequences > 512: 347 (1.44%)
--> Based on this, 23762 sequences (98.56%) will be used with max_seq_length=512.
--- End Analysis ---

Found 24109 valid keys in training H5 file.
Using feature dimension: 208

Initializing tokenizer...
Tokenizer loaded: facebook/mbart-large-50-many-to-many-mmt

--- Creating Validation Splits with STRICT Separation ---
Valid keys after filtering: 23762
Found 16 unique signers and 1889 unique sentences
Standard split: 21385 train, 2377 validation samples
Verified: Train and validation sets have NO overlap
Unseen sentences split: 21388 train, 2374 validation samples
Verified: Train and validation sets have NO overlap
Unseen signers split: 18965 train, 4797 valida

...

Training loss: 0.0308

Evaluating on 'Standard Random Split' validation


Standard Random Split BLEU: 81.51

Evaluating on 'Unseen Sentences Split' validation
Unseen Sentences Split BLEU: 87.99

Evaluating on 'Unseen Signers Split' validation
Unseen Signers Split BLEU: 88.03

Evaluating on 'Test-Like Challenge Split' validation
Test-Like Challenge Split BLEU: 41.97

Generating predictions for 'Official Test Set' validation
Model saved to /content/drive/MyDrive/saudi/output/best_model
New best model saved with BLEU: 41.97

--- Epoch 3/15 ---


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.058000,2.453705,35.963100,11.192200


KeyboardInterrupt: 